<a href="https://colab.research.google.com/github/u2takey/u2takey.github.io/blob/master/coderun2018.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 概念和理论

## 主定理
![定理](https://note.youdao.com/yws/public/resource/d8656d8508f9e26258de7a9bab2ed8f7/xmlnote/D24BDDDF84F74784919C228C1DCBE92A/19392)

https://zh.wikipedia.org/wiki/%E4%B8%BB%E5%AE%9A%E7%90%86


主定理说明的是对于一个递推公式：`T(n) = a*T(n/b) + f(n)`, 时间复杂度`T(n)`是如何被n^logba 和 f(n) 影响的.
http://blog.lirui.me/posts/d53d7104/
```
# 记忆
对于T(n) = a*T(n/b) + f(n), 令：x = n^logba
1. f(n)(多项式) < x => T(n) = Θ(x)
2. f(n)        = x => T(n) = Θ(x*lgn)
3. f(n) (多项式)> x => T(n) = Θ(f(n))
```



# 基础数据结构



In [0]:
# 这部分放所有下面的示例代码中用的数据结构utils函数
class TreeNode:
  def __init__(self, x, left=None, right=None):
    self.val = x
    self.left = left
    self.right = right
    
# Definition for singly-linked list.
class ListNode:
    def __init__(self, x, next=None):
        self.val = x
        self.next = next

# Input:
# [
#   1->4->5,
#   1->3->4,
#   2->6
# ]

def buildLists():
  n11 = ListNode(5)
  n12 = ListNode(4, n11)
  n13 = ListNode(1, n12)
  
  n21 = ListNode(4)
  n22 = ListNode(3, n21)
  n23 = ListNode(1, n22)
  
  n31 = ListNode(6)
  n32 = ListNode(2, n31)

def printList(node):
  while node:
    print(node.val, end=' ')
    node = node.next

## 栈

### 例子：转逆波兰式
波兰式(后缀表达式)是算算术表达式的很好的办法。

需要用两个栈一个符号栈s1, 一个数字栈s2
遍历方式为：

1.   数字 -> s2
2.   符号:
  1.   为 ')' 则 s1 pop到 '(' 到 s2, 为"(" 则 ->s1
  2.   为其他符号则和s1.top()比较优先级，优先级高->s1, 优先级低则一直pop -> s2 直到优先级比s1.top() 高或者相等
3. 遍历完成后 s1.pop所有-> s2

In [0]:
def polish(input):
    # 注意"("的情况需要在if 和 priority都做处理
    def priority(op):
        d = {"*":2, "/":2, "+":1, "-":1, "(" :0}
        return d[op]
    
    s1, s2 = [], []
    for a in input:
        if a.isdigit(): s2.append(a)
        elif a == "(": s1.append(a)
        elif a == ")":
            while s1 and s1[-1] != "(":
                s2.append(s1.pop())
            s1.pop(-1)
        else:
            while s1 and priority(s1[-1])>priority(a):
                s2.append(s1.pop())
            s1.append(a)
    for b in reversed(s1):
        s2.append(b)
    print(s2)
  

In [11]:
polish('1+(2+3)*4')

['1', '2', '3', '+', '4', '*', '+']


### 例子：132 Pattern
https://leetcode.com/problems/132-pattern/description/

In [0]:
class Solution:
    def find132pattern(self, nums):
        """
        :type nums: List[int]
        :rtype: bool
        """
        # 这题的关键是反向遍历 132 如果正向遍历，最后面那个既不是最大也不是最小，很难处理
        # 贪心：s1 < s3 < s2; 如果遍历s1 那么s2取s1 左边最大的，s3取s2左边而且比s2小的最大的
        # 使用 stack 记录"最大", 找s1 < s3, 因为stack里面始终放大的，那么s3存在，就存在s2
        stack = []
        s3 = float("-inf")
        for a in reversed(nums):
            s1 = a
            if s1 < s3:return True
            else:
                while stack and a > stack[-1]:
                    s3 = stack.pop()
                stack.append(a)
        return False

## 最小/最大堆

### 例子：最小堆的实现
http://128kj.iteye.com/blog/1728555

http://interactivepython.org/courselib/static/pythonds/Trees/BinaryHeapImplementation.html

最小堆的操作
0. **上浮**：swap(father, son); **下沉**：swap(father, min of son)
1. 增：append, 上浮
2. 删：替换尾，下沉
3. 查（最小）：顶
4. 建堆：从 n/2->1（这里采用了0为guard的方式）, 下沉


In [0]:
class MinHeap:
    def __init__(self):
        self.heap = [0]
    
    # 上浮
    def up(self, i):
      while i//2 > 0 and self.heap[i//2] > self.heap[i]:
        self.heap[i//2], self.heap[i] = self.heap[i], self.heap[i//2]
        i = i //2
    
    # 下沉
    def down(self, i):
      while i*2 < len(self.heap):
        # 最小堆下沉到小的那个儿子
        son = i*2 
        if i*2+1 < len(self.heap) and self.heap[i*2+1] < self.heap[i*2]:
          son = i*2+1
        if self.heap[i] > self.heap[son]:
          self.heap[son], self.heap[i] = self.heap[i], self.heap[son]
        i = son
    
    # 放在最后，然后上浮
    def push(self, k):
      self.heap.append(k)
      self.up(len(self.heap) - 1)
    
    # 用最后的一个元素替换，然后下沉这个元素
    def delete(self, i):
      self.heap[i], self.heap[-1] = self.heap[-1], self.heap[i]
      self.heap.pop()
      self.down(i)
    
    # 从n/2->1; 依次下沉
    def buildHeap(self, alist):
      self.heap = [0] + alist[:]
      for i in reversed(range(1, len(self.heap))):
        self.down(i)
    
    def p(self):
      print(self.heap)

a = MinHeap()
a.push(2)
a.push(1)
a.push(4)
a.push(5)
a.push(3)
a.p()

a.delete(1)
a.p()

b = MinHeap()
b.buildHeap([2,1,4,5,3])
b.p()                     

### 例子：Merge k Sorted Lists
https://leetcode.com/problems/merge-k-sorted-lists/solution/

这个问题里面有个找最小的操作，如果是比较找最小，那么复杂度是K，而如果是最小堆 插入删除操作时间复杂度是O(lgK)

### 例子：Find Median from Data Stream
https://leetcode.com/problems/find-median-from-data-stream/description/
这道题目的技巧在于把数字分成两个堆来存，保证两个堆长度相同（最多差1），并且两个堆的顶分别为两个中位数

In [0]:
from heapq import *

class MedianFinder:

    def __init__(self):
        self.small, self.large = [], []
    #234  
    def addNum(self, num):
        #python 中只有最小堆
        # 所以self.large堆顶是最小的数，而self.small堆顶是最大的那个数(因为用负号间接完成的最大堆)
        # 这里的计算只是为了保持两个堆的平衡，而且self.large里面的数量>= self.small里面的数量
        heappush(self.large, num)
        heappush(self.small, -heappop(self.large))
        if len(self.large) < len(self.small):
            heappush(self.large, -heappop(self.small))
    
    def findMedian(self):
        if len(self.large) > len(self.small):
            return self.large[0]
        else:
            return (self.large[0] - self.small[0])/2

### 例子：窗口下的最大值

寻找某个限定条件下的最大、最小值常用的两种解法：
1. 用array，array中维护某种关系，遍历的时候去掉不满足条件的数字
2. 用heap，遍历的时候检查heap[0]是否满足条件

例子：寻找数组[4,3,45,4,3,3,6,7]在window k=3的时候的最大值 [5,5,5,4,6,7]


In [0]:
# 解法1:用array
def find_max_in_window_1(l, k):
    ret, q = [], []
    # q中存(number, index)
    for i, a in enumerate(l):
        while q and a >= q[-1][0]:
            q.pop()
        q.append((a, i))
        if q and i - q[0][1] >= k:
            q.pop(0)
        if i >= k-1:
            ret.append(q[0][0])
    return ret

# 解法2:用heapq
import heapq
def find_max_in_window_2(l, k):
    ret, q = [], []
    # q中存(number, index)
    for i, a in enumerate(l):
        while q and i - q[0][1] >= k:
            heapq.heappop(q)
        heapq.heappush(q, (-a, i))
        if i >= k-1:
            ret.append(-q[0][0])
    return ret

## 链表

### 链表问题的技巧
1. dummy node
2. 快慢指针，两个指针，步长不一样，比如 判断有环: 快是慢的两倍，能相遇则有环。求中间的node，快是慢的两倍，快到尾，则慢到中间


---


### 链表相交的问题: 
1. 无环相交，找交点 : 找到 end, end一样则相交；看到end走了多少步m, n,那么 长的先走m-n步，然后同时走, 第一次相遇就是交点 
2. 有环相交，找交点:
  1. 先找入环点loop_entry: 用快慢指针判断有环，第一次相遇之后fast 回到head, 改为一步走，再次相遇就是loop_entry
  2. 如果loop_entry1 = loop_entry2, 那么同问题1
  3. 如果loop_entry1 != loop_entry2, 那么固定loop_entry2, loop_entry1那边继续走一圈，相遇则相交，loop_entry1/2都算交点。如果不相遇，那么不相交，各自有环


关于上面的2.1如何证明：
```python
Consider the following linked list, where E is the cylce entry and X, the crossing point of fast and slow.
H: distance from head to cycle entry E
D: distance from E to X
L: cycle length
                  _____
                 /     \
head_____H______E       \
               _\|       /
                 X_____/
 slow 走了 H + D + i*L; fast 走了  H + D + j *L = 2(H + D + i*L)
 =>  H + D = nL    =>   H = nL - D
 =>  根据现在的位置可以看出，如果 一个从Head走H，另一个从X走H，那么两者肯定会在E处相遇
 
```


### 例子：Palindrome Linked List

https://leetcode.com/problems/palindrome-linked-list/description/



In [0]:
# Definition for singly-linked list.
# class ListNode(object):
#     def __init__(self, x):
#         self.val = x
#         self.next = None

class Solution(object):
    def isPalindrome(self, head):
        """
        :type head: ListNode
        :rtype: bool
        """
        # 1. find middle
        prev = None
        slow = fast = head
        while fast and fast.next:
            fast = fast.next.next
            
            # 同时逆转
            tmp = prev
            prev, slow = slow, slow.next
            prev.next = tmp
            #prev, prev.next, slow = slow, prev, slow.next
        
        if fast:
            slow = slow.next
            
        # 2. compare
        while prev and prev.val == slow.val:
            slow = slow.next
            prev = prev.next
        return prev == None

## 重要：树的分类、概念和性质
树的分类和形式

|分类|是否是二叉树| 是否是查找树 | 是否平衡 | 其他性质
|---|--|--|--|
|树|否|否|否|叶子出度都是0，除了根入度都为1，总入度等于总出度（一般度指出度）
|二叉树|是| 否| 否| 节点（出）度最大为2
|完全二叉树|是|否|否|除了最后一层外，其它各层的节点数目均已达最大值，可用数组存：0位置如果放guard，那么parent_index = son_index // 2
|二叉搜索树|是|是|否|各节点值不同，并且对于任意一个子树：左<根<右
|平衡二叉查找树/AVL树|是|是|是|任何节点的两棵子树的高度差不大于1的二叉树|
|红黑树|是|是|是|和AVL树的不同之处在于：对平衡的要求没有AVL树高
|B树|否|是|是| 平衡多路查找树
|B+树|否|是|是| 是B树的一种变形：非叶结点索引，数据存在叶结点中，所有叶结点构成一个有序链表

### AVL树
1. 查找树都有的性质：左子树值 < root < 右子树值，利用这个性质可以二分找值
2. 查找：最大值：递归最右；最小值：递归最左
3. 插入：找到位置 =》 Balance
4. 删除：找到节点 =》bst删除操作（无儿子直接删；有一个儿子则用儿子替换；有两个儿子则用前驱或者后继来替换） =》 Balance  http://alrightchiu.github.io/SecondRound/binary-search-tree-sortpai-xu-deleteshan-chu-zi-liao.html
5. Balance：LL，RR，LR，RL

### 例子：Verify Preorder Serialization of a Binary Tree
https://leetcode.com/problems/verify-preorder-serialization-of-a-binary-tree/description/
这道题可以
1. 利用树的性质来解：即入度出度之和为0
2. 用stack
3. 递归恢复这个树

## 字典树（前缀树）
主要使用场景：
1. 词频统计：比hash表省空间
2. 前缀匹配：比如查询所有以"a"开头的字符串（或个数）

![替代文字](https://github.com/julycoding/The-Art-Of-Programming-By-July/blob/master/ebook/images/8/8.4/1.jpg?raw=true)


### 例子：前缀树实现
https://leetcode.com/problems/implement-trie-prefix-tree/description/

比较重要的是表达TrieNode的结构
最方便和用途最广的是下面这种结构
```python
Node{
    int path  // 经过这个Node的数量
    int end   // 以这个Node结尾的数量
    map[string]Node children // key是children的字母，children这里也可以用list，因为字母只有26个
}

```


In [0]:
# 这个实现里面把Trie作为TrieNode
import collections
class Trie(object):

    def __init__(self):
        """
        Initialize your data structure here.
        """
        self.path = 0
        self.end = 0
        self.children = collections.defaultdict(Trie)
        

    def insert(self, word):
        """
        Inserts a word into the trie.
        :type word: str
        :rtype: void
        """
        node = self
        for a in word:
            node = node.children[a]
            node.path += 1
        node.end += 1

    def search(self, word):
        """
        Returns if the word is in the trie.
        :type word: str
        :rtype: bool
        """
        node = self
        for a in word:
            node = node.children[a]
        return node.end != 0
        

    def startsWith(self, prefix):
        """
        Returns if there is any word in the trie that starts with the given prefix.
        :type prefix: str
        :rtype: bool
        """
        node = self
        for a in prefix:
            node = node.children[a]
        return node.path != 0
        


# Your Trie object will be instantiated and called as such:
# obj = Trie()
# obj.insert(word)
# param_2 = obj.search(word)
# param_3 = obj.startsWith(prefix)

### 例子：Maximum XOR of Two Numbers in an Array
https://leetcode.com/problems/maximum-xor-of-two-numbers-in-an-array/description

对于一个数10001 就是从前到后找和它字符不同的，也就是说找一个前缀为01110的（如果存在），如果不存在，也可以用这个思路找尽量大的。用这个思路就可以建一个前缀数，把所有的数据放进去。

In [0]:
class TrieNode:
    def __init__(self, val=-1):
        self.left = None  # 0
        self.right = None # 1
        self.val = val
        
    def add(self, val):
        # self as root
        node = self
        for i in reversed(range(32)):
            if val & (1<<i):
                if not node.right:
                    node.right = TrieNode()
                node = node.right
            else:
                if not node.left:
                    node.left = TrieNode()
                node = node.left
        node.val = val
                
    def findMaxDiff(self, val):
        node = self
        for i in reversed(range(32)):
            if (val & (1<<i) and node.left) or not node.right:
                node = node.left
            else:
                node = node.right
        return node.val
        

class Solution:
    def findMaximumXOR(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        # 前缀树
        root = TrieNode()
        maxxor = 0 
        for a in nums:
            root.add(a)
            b = root.findMaxDiff(a)
            maxxor = max(maxxor, b^a)
        return maxxor

### 例子：Prefix and Suffix Search

https://leetcode.com/problems/prefix-and-suffix-search/description/

        #两个技巧
        #技巧1，创造出可以查询 f("a", "e") 这样的单词, 如apple 增加 #apple e#apple el#apple elp#apple  elpp#apple elppa#apple
        #技巧2，因为返回最大weight, 从前到后迭代覆盖weight节点weight即可

In [0]:
import collections
class Trie:
    def __init__(self):
        self.children = collections.defaultdict(Trie)
        self.weight = 0
        
    def add(self, word, weight):
        n = self
        n.weight = weight
        for w in word:
            n = n.children[w]
            n.weight = weight
    
    def search(self, prefix):
        n = self
        for w in prefix:
            if w in n.children:
                n = n.children[w]
            else:
                return -1
        return n.weight
        

class WordFilter(object):

    def __init__(self, words):
        """
        :type words: List[str]
        """
        self.root=Trie()
        for weight, w in enumerate(words):
            for i in range(len(w)+1):
                new_w = w[i:]+ "#" + w
                self.root.add(new_w, weight)
        

    def f(self, prefix, suffix):
        """
        :type prefix: str
        :type suffix: str
        :rtype: int
        """
        #两个技巧
        #技巧1，创造出可以查询 f("a", "e") 这样的单词, 如apple 增加 #apple e#apple el#apple elp#apple  elpp#apple elppa#apple
        #技巧2，因为返回最大weight, 从前到后迭代覆盖weight节点weight即可
        if prefix == "" and suffix == "":
            return 0
        new_prefix = suffix + "#" + prefix
        return self.root.search(new_prefix)
        


# Your WordFilter object will be instantiated and called as such:
# obj = WordFilter(words)
# param_1 = obj.f(prefix,suffix)

## 线段树/Segment Tree

线段树（segment tree），是用来存放给定区间（segment, or interval）内对应信息的一种数据结构（比如每个区间的最小值，最大值，区间和等）如果是数组，区间就是index (i, j)  为一个区间

```python
# 这样一种树形区间结构，能够吧区间问题变成binary search的问题
|----------------------|
|----------||----------|
|----||----||----||----|
.....

```

https://leetcode.com/problems/range-sum-query-mutable/solution/
这里是一个利用循环的实现
使用递归实现更好理解和记忆

https://www.geeksforgeeks.org/segment-tree-set-1-sum-of-given-range/

**重要：记住线段树的实现方式2（class实现）、1（数组实现）**

应用：
https://leetcode.com/problems/number-of-longest-increasing-subsequence/description/

In [0]:
# 实现1，用数组存

import math
class SegmentTree(object):
    def __init__(self, nums):
        self.n = len(nums)
        if self.n == 0:
            return
        self.nums = nums
        x = math.ceil(math.log(self.n, 2))
        self.segment_tree = [0] * (2**int(x+1))
        self.build_tree(nums, 0, 0, self.n-1)
        
    # node为节点，因为使用数组存的，所以node为i, 左儿子为2n+1, 右儿子为2n+2
    # nstart, nend为node表示的区间
    def build_tree(self, nums, node, nstart, nend):
        if nstart == nend:
            # 叶子
            self.segment_tree[node] = nums[nstart]
        else:
            mid = (nstart + nend) // 2
            # build 左右子树
            l = self.build_tree(nums, node*2+1, nstart, mid)
            r = self.build_tree(nums, node*2+2, mid+1, nend)
            # 因为这里线段树是为了求和设计的，所以这里存区间和
            self.segment_tree[node] = l + r
        return self.segment_tree[node] 
    
    def update(self, pos, val):
        diff = val - self.nums[pos]
        self.nums[pos] = val
        self._update(0, 0, self.n-1, pos, diff)

    #  pos为在数组中的位置
    #  node为节点
    #  nstart, nend为node表示的区间
    def _update(self, node, nstart, nend, pos, diff):
        if pos < nstart or pos > nend:
            return
        self.segment_tree[node] += diff;
        if nstart != nend:
            mid = (nstart + nend) // 2
            self._update(node*2+1, nstart, mid, pos, diff)
            self._update(node*2+2, mid+1, nend, pos, diff)
    
    def query(self, qstart, qend):
        return self._query(0, 0, self.n-1, qstart, qend)
    
     #  node：为节点
     #  [nstart, nend]: node所表示的区间
     #  [qstart, qend]: 查询的区间     
    def _query(self, node, nstart, nend, qstart, qend):
        if qstart > nend or qend < nstart:
            return 0
        elif qstart <= nstart and qend >= nend:
            return self.segment_tree[node]
        else:
            mid = (nstart+nend)//2
            l = self._query(node*2+1, nstart, mid, qstart, qend)
            r = self._query(node*2+2, mid+1, nend, qstart, qend)
            return l + r
    
class NumArray(object):

    def __init__(self, nums):
        """
        :type nums: List[int]
        """
        self.nums = nums
        self.segment_tree = SegmentTree(nums)
        
    
    def update(self, i, val):
        """
        :type i: int
        :type val: int
        :rtype: void
        """
        self.segment_tree.update(i, val)
        

    def sumRange(self, i, j):
        """
        :type i: int
        :type j: int
        :rtype: int
        """
        return self.segment_tree.query(i, j)

In [0]:
# 实现2，用class存
# 同时参考这一题的实现（Lazy SegmentTree) https://leetcode.com/problems/my-calendar-iii/description/
class SegmentTreeNode(object):
    def __init__(self, start, end, left=None, right=None, s=0):
        self.sum = s
        self.start, self.end, self.left, self.right= start, end, left, right
        
def buildSegmentTree(nums, start, end):
    if start > end:
        return None
    elif start == end:
        return SegmentTreeNode(start, end, None, None, nums[start])
    else:
        mid = (start + end) //2
        l = buildSegmentTree(nums, start, mid)
        r = buildSegmentTree(nums, mid+1, end)
        return SegmentTreeNode(start, end, l, r, l.sum + r.sum)
    
def updateSegmentTree(root, pos, val):
    if root.start == root.end:
        root.sum = val
    else:
        mid = (root.start + root.end) //2
        if pos <= mid:
            updateSegmentTree(root.left, pos, val)
        else:
            updateSegmentTree(root.right, pos, val)
        root.sum = root.left.sum + root.right.sum 
        
def querySegmentTree(root, start, end):
    if not root or start > root.end or end < root.start:
        return 0
    elif start <= root.start and end >= root.end:
        return root.sum
    else:
        mid = (root.start + root.end) //2
        if end <= mid:
            return querySegmentTree(root.left, start, end);
        elif start >= mid+1: 
            return querySegmentTree(root.right, start, end);
        else:   
            return querySegmentTree(root.right, mid+1, end) + querySegmentTree(root.left, start, mid);
    
    
class NumArray(object):

    def __init__(self, nums):
        """
        :type nums: List[int]
        """
        self.segment_tree = buildSegmentTree(nums, 0, len(nums)-1)
        
    
    def update(self, i, val):
        """
        :type i: int
        :type val: int
        :rtype: void
        """
        updateSegmentTree(self.segment_tree, i, val)
        

    def sumRange(self, i, j):
        """
        :type i: int
        :type j: int
        :rtype: int
        """
        return querySegmentTree(self.segment_tree, i, j)
        

In [0]:
# 实现3: https://leetcode.com/problems/range-sum-query-mutable/solution/

### 例子：My Calendar III

https://leetcode.com/problems/my-calendar-iii/description/

这道题同My Calendar II，实际不用segment tree 实现，可以用下面的bisect方法解决，但是这里是一个演示使用segment tree的一个很好的例子，值得记住segment tree的解法

附My Calendar II的解法:
```python
#My Calendar II的题目也不错，但是和线段树无关，这里贴一下代码
#https://leetcode.com/problems/my-calendar-ii/description/
import bisect
class MyCalendarTwo:

    def __init__(self):
        self.calendar = []
        self.overlaps = []
    
    # 方法1: overlap1次的存一个数组，用于第二次比较
    def book1(self, start, end):
        for i, j in self.overlaps:
            if start < j and end > i:
                return False
        for i, j in self.calendar:
            if start < j and end > i:
                self.overlaps.append((max(start, i), min(end, j)))
        self.calendar.append((start, end))
        return True
    
    # 方法2：start了3次的有问题，就不插入了(使用tmp), 这个方法扩展性很好
    def book(self, start, end):
        tmp = list(self.calendar)
        bisect.insort(tmp, (start, True))
        bisect.insort(tmp, (end, False))
        c = 0
        for a, isStart in tmp:
            if isStart:
                c += 1
                if c >= 3:
                    return False
            else:
                c -= 1
        
        self.calendar = tmp
        return True
```



In [0]:
class SegNode:
    # start, end是区间, left, right是左右儿子
    def __init__(self, start, end, left=None, right=None):
        self.start, self.end = start, end
        self.left, self.right = left, right
        self.booked = 0
        self.maxIntersects = 0
    
    def add(self, start, end):
        if start <= self.start and end >= self.end:
            self.booked += 1
            self.maxIntersects += 1
            return 
        # 说明有重叠
        if not (start >= self.end or end <= self.start):
            mid = (self.start + self.end) // 2
            if self.left == None:
                self.left = SegNode(self.start, mid)
            self.left.add(start, end)
            if self.right == None:
                # 为什么这里mid不加1? 因为 start, end已经是左开右闭区间
                self.right = SegNode(mid, self.end)
            self.right.add(start, end)
            self.maxIntersects = max(self.left.maxIntersects, self.right.maxIntersects) + self.booked
    
    # 用于展示，最后生成的是一颗什么样的树
    def print(self):
        #bfs
        q = [(self, 1)]
        ll = []
        while q:
            n, l = q.pop(0)
            if l > len(ll):
                ll.append([(n.start, n.end)])
            else:
                ll[-1].append((n.start, n.end))
            if n.left:
                q.append((n.left, l + 1))
            if n.right:
                q.append((n.right, l + 1))
        print("----------------------")   
        for l in ll:
            print(l)
        
    
class MyCalendarThree:

    def __init__(self):
        self.root = SegNode(0, 1000000000)

    def book(self, start, end):
        """
        :type start: int
        :type end: int
        :rtype: int
        """
        self.root.add(start, end);
        #self.root.print()
        return self.root.maxIntersects
        
        

        


# Your MyCalendarThree object will be instantiated and called as such:
# obj = MyCalendarThree()
# param_1 = obj.book(start,end)



# bisect解法：
import bisect
class MyCalendarThree:

    def __init__(self):
        self.calendar = []

    def book(self, start, end):
        """
        :type start: int
        :type end: int
        :rtype: int
        """
        bisect.insort(self.calendar, (start, True))
        bisect.insort(self.calendar, (end, False))
        c, maxc = 0, 0
        for t, isStart in self.calendar:
            if isStart:
                c += 1
                maxc = max(c, maxc)
            else:
                c -= 1
        return maxc

## Binary Indexed Tree
https://blog.csdn.net/Yaokai_AssultMaster/article/details/79492190


**对于 Binary Indexed Tree 主要记住如下实现**
1. c数组的index从1开始，c[i] 中存的是 num[i-1] 之前的多个数字的和，具体range长度和i末尾的0的个数有关系
2. 从num -> c, 对于一个num[i] 需要加在 k=i+1; k += lowbit(k); k < len上面，其中lowbit指把保留最后一个1得到的数字，常用k & (-k)
3. 从c -> sum_range(j), 对于一个sum_range(0, j); 需要求和c[k], 其中k = j+ 1; k-=lowbit(k);k > 0; 其中lowbit指把保留最后一个1得到的数字，常用k & (-k)

In [0]:
class NumArray(object):
    def __init__(self, nums):
        self.n = len(nums)
        self.a, self.c = nums, [0] * (self.n + 1)
        for i in range(self.n):
            k = i + 1
            while k <= self.n:
                self.c[k] += nums[i]
                k += (k & -k)

    def update(self, i, val):
        diff, self.a[i] = val - self.a[i], val
        i += 1
        while i <= self.n:
            self.c[i] += diff
            i += (i & -i)

    def sumRange(self, i, j):
        res, j = 0, j + 1
        while j:
            res += self.c[j]
            j -= (j & -j)
        while i:
            res -= self.c[i]
            i -= (i & -i)
        return res

## 哈夫曼（huffman）树和哈夫曼编码
哈夫曼树性质：
1. wpl（加权路径和）最小的树
2. 左右子树还是哈夫曼树

构建树：
循环把权值最小的树合并
https://blog.csdn.net/FX677588/article/details/70767446

```python
Q = minQueue(nodes)
for i in range(1, len):
  node.left = ExtractMin(Q)
  node.right= ExtractMin(Q)
  insert(Q, node)
```

## 数据结构设计


### 例子: Insert Delete GetRandom O(1)

https://leetcode.com/problems/insert-delete-getrandom-o1/description/

**这个题的关键是使用 array + map: map 记录 index, pop的时候 swap(index, tail) 并更新index
这是一个常用的数据结构设计，array删除index非O(1), 把删除转化成pop()，就能避免这个问题**

# 图

## 图的遍历
同树的遍历：bfs, dfs，唯一不同的是比如部分有向图或者无向图，需要加一个visited table记录遍历状态

### 例子：Pacific Atlantic Water Flow

https://leetcode.com/problems/pacific-atlantic-water-flow/description/

这个例子可以虚拟出一个Pacific, Atlantic Node，也可以start就是多个边缘上的节点

### 例子：Concatenated Words
https://leetcode.com/problems/concatenated-words/description/

找出这样的string，他可以由其他至少两个string连接构成（可以重复使用）
Input: ["cat","cats","catsdogcats","dog","dogcatsdog","hippopotamuses","rat","ratcatdogcat"]

Output: ["catsdogcats","dogcatsdog","ratcatdogcat"]



In [0]:

class Solution(object):
    # 可以看成dfs问题或者是dp的问题
    def findAllConcatenatedWordsInADict(self, words):
        words = sorted(words,key=lambda t:len(t))
        d = set()
        def dfs(w):
            if w in d:return True
            for i in range(1,len(w)):
                if w[:i] in d and dfs(w[i:]):
                    return True
            return False
        res = []
        for w in words:
            if dfs(w):
                res.append(w)
            # 这里加w 这样搜索的时候就不用 d-{w}
            d.add(w)
        return res
    

    # 思路2: 看成了图问题：Let's discuss whether a word should be included in our answer.
    # Consider the word as a topologically sorted directed graph, where each node is a letter,
    # and an edge exists from i to j if word[i:j] is in our wordlist, 
    # [and there is no edge from i=0 to j=len(word)-1]. 
    # We want to know if there is a path from beginning to end. If there is,
    # then the word can be broken into concatenated parts from our wordlist. 
    # To answer this question, we DFS over this graph.

## 欧拉回路
有两种欧拉路。第一种叫做 Eulerian path(trail)，沿着这条路径走能够走遍图中每一条边；第二种叫做 Eularian cycle，沿着这条路径走，不仅能走遍图中每一条边，而且起点和终点都是同一个顶点。注意：欧拉路要求每条边只能走一次，但是对顶点经过的次数没有限制。

满足什么性质的图才能有欧拉路？

1. **在无向图中，所有顶点的度数均为偶，则存在 Eularian cycle**；**若有且仅有两个顶点的度数为奇，其余的都为偶，则存在 Eularian path**；

2. **在有向图中，所有顶点的入度数等于出度数，则存在 Eularian cycle**；**若有且仅有两个顶点：其中一个入度数比出度数大 1，另一个入度数比出度数小 1，其余的顶点入度数等于出度数，则存在 Eularian path**.

解法


```python
# 1. 可以看成一种post order的dfs遍历
# 2. 注意这里mark的是edge, 即seen add edge或者pop edge
path = []

DFS(u):
    While (u存在未被访问的边e(u,v))
        mark边e(u,v)为访问
        DFS(v)
    End
    path.append(u)
    
path = path[::-1]

```

### 例子: Reconstruct Itinerary
https://leetcode.com/problems/reconstruct-itinerary/description/

In [0]:
from collections import defaultdict
class Solution:
    def findItinerary(self, tickets):
        """
        :type tickets: List[List[str]]
        :rtype: List[str]
        """
        
        def dfs(u, edges, path):
            while edges[u]:
                d = edges[u].pop(0)
                dfs(d, edges, path)
            path.append(u)
        
        
        edges = defaultdict(list)
        for s, e in sorted(tickets):
            edges[s].append(e)
         
        path = []
        dfs("JFK", edges, path)
        return path[::-1]

### 例子: Cracking the Safe

https://leetcode.com/problems/cracking-the-safe/description/
https://leetcode.com/problems/cracking-the-safe/solution/

抽象这个问题为欧拉回路问题，比如k=4，n=3 那么node为00, 01, 02,... 32, 33, 每个node都有4个边为0,1,2,3 那么一个node+edge就是一个可能解，也是欧拉回路的一个substring。那么我们可以从"00"开始找这欧拉回路

In [0]:
class Solution:
    def crackSafe(self, n, k):
        """
        :type n: int
        :type k: int
        :rtype: str
        """
        #00 11 10 01   011  100 011
        seen = set()
        path = []
        def dfs(node):
            for a in map(str, range(k)):
                candi = node + a
                if candi not in seen:
                    seen.add(candi)
                    # candi[1:] 是next_node
                    dfs(candi[1:])
                    path.append(a)
                    
        dfs("0"*(n-1))
        path.append("0"*(n-1))
        return "".join(path)
                    

## 强连通分支
1. 无向图的连通子图：bfs，dfs搜索即可
2. 有向图的强连通子图：主要有两种算法 https://www.cnblogs.com/bethunebtj/p/4854946.html 记住Kosaraju算法就可以了：
  1. DFS(G). 完成次序（后序）为Q
  2. DFS(G-T). 考虑次序为Q-T
  3. 得到的字树就是强连通子图


在有向图G中，如果两个顶点间至少存在一条路径，称两个顶点强连通(strongly connected)。如果有向图G的每两个顶点都强连通，称G是一个强连通图。非强连通图有向图的极大强连通子图，称为强连通分量(strongly connected components)。

## 最短路径问题

|问题|算法| 复杂度 | 算法描述 |
|---|--|--|--|
|单源无权图|bfs|O(V+E)|queue, table: Dist;Prev
|单源有权图(无负边)|dijkstra|O(Elgv) -> O(E+VlgV)|见下
|单源有权图(可能有负边)|Bellman-Ford|O(EV)| 可以检测到负权环
|多源有/无权图|前面的算法重复n次|O(v^2+VE)~O(EVlgV)|
|多源有/无权图|Floyd算法|O(V^ 3)|这是一个dp算法d[i,j] = min(d[i,k] + d[k, j] for k in connected nodes) 注意k在最外层循环
|多源有/无权图|Jason算法|O(V^2lgV+VE)|一种重新赋权技术，同时也是一种改进的dijkstra，使之适合负环

In [0]:
# 1. dijkstra
# 伪代码
# init 距离矩阵Q, start为0 其他都是inf; 
# Q = V[G]
# While Q
#    u <- ExtratMin(Q)
#    for v in u's neighbors:
#         do relax(u, v, w)


from collections import defaultdict
from heapq import *

def dijkstra(edges, f, t):
    g = defaultdict(list)
    for l,r,c in edges:
        # 存邻接点 
        g[l].append((c,r))
    
    # q用于ExtratMin, mins用于检查是否relax了
    # q中存的格式为 (距离,节点,当前走的路径)
    q, seen, mins = [(0,f,())], set(), {f: 0}
    while q:
        (cost,v1,path) = heappop(q)
        # 这句很关键，这种算法heap中有重复的
        if v1 not in seen:
            seen.add(v1)
            path = (v1, path)
            if v1 == t: return (cost, path)
            # 访问所有邻接点
            for c, v2 in g.get(v1, ()):
                if v2 in seen: continue
                # 看看relax到了多少了，有relax才有必要放进 heap 中去
                # heap中有重复的，但是seen会避免看重复节点的可能性
                prev = mins.get(v2, None)
                next = cost + c
                if prev is None or next < prev:
                    mins[v2] = next
                    heappush(q, (next, v2, path))

    return float("inf")
  

# 2. Bellman-Ford
# Init: 距离矩阵Q，start为0 其他都是inf; 
# for u in V[G]:
#    for edge(u, v) in E[G]:
#       Relax(u, v, w)
# for edge in E[G]:
#   # 还有relax的余地？检测到负权环
#   if d[v] > d[u] + w(u, w):
#     return False



# 3. jason
# 1) 新建一个点S，这个S和图中所有的点连线，距离为0
# 2) 因为有负边，可以用Bellman-Ford算法算出h[u] 表示点S到任意点u的最小距离
# 3) 重新赋权w_new(u, v) = w(u, v) + h[u] - h[v] 这样weight都会>=0
# 4) 用dijkstra算出多源最小距离 d_new
# 5）使用h来恢复距离，d(u, v) = d_new(u, v) + h(v) - h(u)


# 4. floyd
# 1) let dist be a |V| × |V| array of minimum distances initialized to ∞ (infinity)
# 2) for each vertex v
# 3)    dist[v][v] ← 0
# 4) for each edge (u,v)
# 5)    dist[u][v] ← w(u,v)  // the weight of the edge (u,v)
# 6) for k from 1 to |V|
# 7)    for i from 1 to |V|
# 8)       for j from 1 to |V|
# 9)          if dist[i][j] > dist[i][k] + dist[k][j] 
# 10)             dist[i][j] ← dist[i][k] + dist[k][j]
# 11)         end if


### 例子：Evaluate Division

https://leetcode.com/problems/evaluate-division/description/

这个例子可以看成一个求多源路径（Floyd–Warshall）的问题，除数和被除数连线，那么能求解 等价于 这两个点之间有路径，路径的距离可以用除法算出来，由于query次数多，先把多源路径都先算出来也是最合适的办法。

In [0]:
import collections 
class Solution:
    def calcEquation(self, equations, values, queries):
        """
        :type equations: List[List[str]]
        :type values: List[float]
        :type queries: List[List[str]]
        :rtype: List[float]
        """
        edges = collections.defaultdict(dict)
        for i, (src, dst) in enumerate(equations):
            edges[src][dst] = values[i]
            edges[dst][src] = 1/ values[i]
            edges[src][src] = 1
            edges[dst][dst] = 1
        
        # every node
        for k in edges:
            # 这样遍历不用判断边在不在了
            for i in edges[k]:
                for j in edges[k]:
                    edges[i][j] = edges[i][k] * edges[k][j]
        
        return [edges[src].get(dst, -1.0) for src, dst in queries]

## 例子：几种算法实现对比
https://leetcode.com/problems/cut-off-trees-for-golf-event/description/

这个题目里面A*是比较好的算法

A*搜索算法，俗称A星算法。这是一种在图形平面上，有多个节点的路径，求出最低通过成本的算法。常用于游戏中的NPC的移动计算，或网络游戏的BOT的移动计算上。

该算法综合了Best-First Search和Dijkstra算法的优点：在进行启发式搜索提高算法效率的同时，可以保证找到一条最优路径（基于评估函数）。

在此算法中，如果以  g(n)表示从起点到任意顶点  n的实际距离，  h(n)表示任意顶点  n到目标顶点的估算距离（根据所采用的评估函数的不同而变化），那么A*算法的估算函数为：

** f(n)=g(n)+h(n)**

这个公式遵循以下特性：

1. 如果 g(n)为0，即只计算任意顶点 n到目标的评估函数 h(n)，而不计算起点到顶点n的距离，则算法转化为使用贪心策略的Best-First Search，速度最快，但可能得不出最优解；
2. 如果 h(n)不大于顶点 n到目标顶点的实际距离，则一定可以求出最优解，而且 h(n)越小，需要计算的节点越多，算法效率越低，常见的评估函数有——欧几里得距离、曼哈顿距离、切比雪夫距离；
3. 如果 h(n)为0，即只需求出起点到任意顶点 n的最短路径 g(n)，而不计算任何评估函数 h(n)，则转化为单源最短路径问题，即Dijkstra算法，此时需要计算最多的定点；


In [0]:
import itertools
import collections
import heapq
class Solution(object):
    def __init__(self):
        self.d = {}
        
    def dist(self, forest, start, end):
        # return self.floyd(forest, start, end)
        # return self.bfs(forest, start, end)
        # return self.dijkstra(forest, start, end)
        return self.hadlocks(forest, start, end)
    
    # 1. 使用floyd 算距离，耗时比较长，因为会把很多没必要的距离也算出来
    def floyd(self, forest, start, end):
        if self.d:
            return self.d[(start, end)]
        def default():
            return -1
        m, n = len(forest), len(forest[0])
        v = list(a for a in itertools.product(range(m), range(n)) if forest[a[0]][a[1]] != 0)
        self.d = collections.defaultdict(default)
        for i in range(m):
            for j in range(n):
                if forest[i][j] != 0:
                    self.d[((i, j), (i, j))] = 0
                    for (a, b) in {(i-1, j), (i+1, j), (i, j-1), (i, j+1)}:
                        if 0 <= a < m and 0 <=b < n and forest[a][b] != 0:
                            self.d[((a, b), (i, j))] = 1
                            self.d[((i, j), (a, b))] = 1
        for k in v:
            for a in v:
                for b in v:
                    self.d[(a, b)] = min(self.d[(a, b)], self.d[(a, k)] + self.d[(k, b)])
        return self.d[(start, end)]
    
    # 2. 使用bfs算距离，耗时也比较长，比floyd略好
    def bfs(self, forest, start, end):
        
        m, n = len(forest), len(forest[0])
        queue = collections.deque([(start, 0)])
        seen = {start}
        while queue:
            node, d = queue.popleft()
            if node == end:
                return d
            i, j = node[0], node[1]
            for (a, b) in {(i-1, j), (i+1, j), (i, j-1), (i, j+1)}:
                if 0 <= a < m and 0 <=b < n and forest[a][b] != 0 and (a, b) not in seen:
                    seen.add((a, b))
                    queue.append(((a, b), d+1))
        return -1
    
    # 3. 使用dijkstra算法
    def dijkstra(self, forest, start, end):
        m, n = len(forest), len(forest[0])
        heap = [(0, start)] # dist, node
        mindist, seen = {start: 0}, {start}
        while heap:
            dist, node = heapq.heappop(heap)
            seen.add(node)
            if node == end: 
                return dist
            i, j = node
            for (a, b) in {(i-1, j), (i+1, j), (i, j-1), (i, j+1)}:
                if 0 <= a < m and 0 <=b < n and forest[a][b] != 0 and (a, b) not in seen:
                    new_dist = dist + 1
                    if new_dist < mindist.get((a, b), 9999):
                        mindist[(a, b)] = new_dist
                        heapq.heappush(heap, (new_dist, (a, b)))
                        
        return -1
            
    
    # 4. 使用A*算法，A*使用一套启发估算，并且当估算函数合适的时候肯定能找到最优解
    #    使用A*算法不需要seen, A*需要遍历的点比 dijkstra 可能少很多
    #     https://qiao.github.io/PathFinding.js/visual/
    #    https://zh.wikipedia.org/wiki/A*%E6%90%9C%E5%B0%8B%E6%BC%94%E7%AE%97%E6%B3%95
    #    dijkstra 可以看成A*算法的特殊情况(h(v) = 0) 
    #    https://stackoverflow.com/questions/13031462/difference-and-advantages-between-dijkstra-a-star
    
    def astar(self, forest, start, end):
        if (start, end) in self.d:
            return self.d[start, end]
        m, n = len(forest), len(forest[0])
        mincost = {start:0}
        q = [(0, 0, start)] # cost, dist, node
        while q:
            cost, dist, node = heapq.heappop(q)
            self.d[start, node] = dist
            if node == end:
                return dist
            i, j = node
            for a, b in {(i-1, j), (i+1, j), (i, j-1), (i, j+1)}:
                if 0 <= a < m and 0 <= b < n and forest[a][b] != 0:
                    cost = dist + 1 + abs(end[0] - a) + abs(end[1] - b)
                    if cost < mincost.get((a, b), float("inf")):
                        mincost[a, b] = cost
                        heapq.heappush(q, (cost, dist+1, (a, b)))
        return -1
    
    # 5. hadlocks算法：这是grid里面算路的一种特殊算法，detours是偏离target的距离
    # dist(source, target) = taxi(source, target) + 2 * detours
    # 比如x，y方向偏离了target方向，那么把这一步增加一个detours
    def hadlocks(self, forest, start, end):
        sr, sc = start
        tr, tc = end
        R, C = len(forest), len(forest[0])
        processed = set()
        deque = collections.deque([(0, sr, sc)])
        while deque:
            detours, r, c = deque.popleft()
            if (r, c) not in processed:
                processed.add((r, c))
                if r == tr and c == tc:
                    return abs(sr-tr) + abs(sc-tc) + 2*detours
                for nr, nc, closer in ((r-1, c, r > tr), (r+1, c, r < tr),
                                       (r, c-1, c > tc), (r, c+1, c < tc)):
                    if 0 <= nr < R and 0 <= nc < C and forest[nr][nc]:
                        if closer:
                            deque.appendleft((detours, nr, nc))
                        else:
                            deque.append((detours+1, nr, nc))
    # 本质是算dist，算dist这里距离了4种算法
    def cutOffTree(self, forest):
        """
        :type forest: List[List[int]]
        :rtype: int
        """
    
        m, n = len(forest), len(forest[0])
        queue = sorted((forest[i][j], (i, j)) for i in range(m) for j in range(n) if forest[i][j] > 1)
        p = (0, 0)
        c = 0
        for _, n in queue:
            dist = self.dist(forest, p, n)
            if dist < 0:
                return -1
            c += dist
            p = n
                    
        return c
        
        

## 最小生成树
最小生成树是一副连通加权无向图中一棵权值最小的生成树
https://zh.wikipedia.org/wiki/%E6%9C%80%E5%B0%8F%E7%94%9F%E6%88%90%E6%A0%91

主要有两种算法Prim算法和Kruskal算法


Prim算法(让树成长，找距离树最近的边)
```python
1).输入：一个加权连通图，其中顶点集合为V，边集合为E；
2).初始化：Vnew = {x}，其中x为集合V中的任一节点（起始点，Enew = {},为空；
3).重复下列操作，直到Vnew = V：
  a.在集合E中选取权值最小的边<u, v>，其中u为集合Vnew中的元素，而v不在Vnew集合当中，并且v∈V（如果存在有多条满足前述条件即具有相同权值的边，则可任意选取其中之一）；
  b.将v加入集合Vnew中，将<u, v>边加入集合Enew中；
4).输出：使用集合Vnew和Enew来描述所得到的最小生成树。

```

Kruskal算法(森林合并, 总拿最小的边但是和已经拿的边不构成回路)
```python
1).记Graph中有v个顶点，e个边
2).新建图Graphnew，Graphnew中拥有原图中相同的e个顶点，但没有边
3).将原图Graph中所有e个边按权值从小到大排序
4).循环：从权值最小的边开始遍历每条边 直至图Graph中所有的节点都在同一个连通分量中
  if 这条边连接的两个节点于图Graphnew中不在同一个连通分量中:
    添加这条边到图Graphnew中

```
Kruskal算法里面会用到并查集算法

```python
# a. make_set
def make_set(x): 
  father(x) = x
# b. find_set
def find_set(x): 
  while(x != father(x)): 
    x = father(x)
# c. union_set
def union_set(x,y):
  xf, yf =find_set(x), find_set(y), 
  if xf == yf:
    return 
  else:
    father(xf) = yf
  
```



## 拓扑排序
拓扑排序主要有两种常见的解决方法
1. Kahn算法：优先使用的办法，可以检测有环的情况（即：不需要输入为DAG），**所以这种算法也常用来检测图有无环**  (**网上对于这种也有‘dfs’, ‘bfs’ 实现，但是其实pop的时候不关心次序，所以在这种算法实现下面的dfs, bfs并没有必要区分**)
2. 基于dfs的算法：检测有环情况比较麻烦，需要在dfs的时候同时判断有无环


---

```python
// Kahn算法算法描述
L ← 用来存拓扑排序结果
S ← 入度为0的节点集合, 先要算一下入度，把入度为0的先放进S
// 这里入度的统计也可以直接用edge_dict{edge: in_edge_list}来表示，这样同时可以表示入度以及入边列表

while S:
    n = S.pop()
    L.append(n)
    for m in n's neighbors( 有边 n->m): 
        m的入度 -=1，总入度 -= 1，如果m的入度==0: S.append(m)
if 总入度不为0：
    有环推出
else 
    return L
    
// 基于dfs的算法描述
L ← 用来存拓扑排序结果
S ← 入度为0的节点集合, 先要算一下入度，把入度为0的先放进S
for each node n in S：
    dfs(n) 
dfs的每个node的结束时append进L
把L逆序（即：拓扑排序结果和dfs完成时间的次序相反）
```

### 例子:  Course Schedule
https://leetcode.com/problems/course-schedule/description/

In [0]:
import collections
class Solution:
    def canFinish(self, numCourses, prerequisites):
        """
        :type numCourses: int
        :type prerequisites: List[List[int]]
        :rtype: bool
        """
        dgrees = collections.defaultdict(int)
        edges = collections.defaultdict(list)
        totaldgree, s = 0, set(range(0, numCourses))
        for e in prerequisites:
            source, dest = e
            edges[source].append(dest)
            dgrees[dest] += 1
            totaldgree += 1
        for k in dgrees.keys():
            s.remove(k)
        while s:
            n = s.pop()
            # do print
            for m in edges[n]:
                dgrees[m] -= 1
                totaldgree -= 1
                if dgrees[m] == 0:
                    s.add(m)
        return totaldgree == 0

### 例子: Minimum Height Trees
https://leetcode.com/problems/minimum-height-trees/description/

这道题并不是拓扑排序的题，但是类似拓扑排序的思路：找一个node，以这个node作为root，总degree最小。
degree是一个点的边，那么可以看出leaf的degree总是1，也是树中dgree最小的node，那么可以类似一种bfs（从外到内）或者拓扑排序的思路，从leaf开始遍历，在逐渐剔除-》产生新的leaf，那么剩下的最后一个或者二个节点就是所有找的点了。

（这道题也可以用多源最短路径来解，但是时间复杂度是O(n^3)）

In [0]:
import collections
class Solution:
    def findMinHeightTrees(self, n, edges):
        """
        :type n: int
        :type edges: List[List[int]]
        :rtype: List[int]
        """
        if n == 1: return [0]
        d = collections.defaultdict(set)
        for s, e in edges:
            d[s].add(e)
            d[e].add(s)
            
        leafs = [s for s in d.keys() if len(d[s]) == 1]
        while n > 2:
            n -= len(leafs)
            newleafs = []
            for s in leafs:
                e = d[s].pop() # 只可能有一个
                d[e].remove(s)
                if len(d[e]) == 1:
                    newleafs.append(e)
            leafs = newleafs
        return leafs
            
            

# 排序

https://www.cnblogs.com/alsf/p/6606287.html

|算法|平均时间复杂度| 最坏 | 额外空间 |稳定 |备注
|---|--|--|--|--|--|
|简单选择|N^2|N^2|1|X |依次选择最大的
|冒泡|N^2|N^2|1|Y|依次比较相邻元素
|插入|N^2|N^2|1|Y|依次从后往前把元素插入到合适位置
|希尔|N^d(根据gap, d可能为1.5，1.25)|N^2|1|X| gap,N/2 ->1多次插入
|堆|NlgN|NlgN|1|X|建堆，出堆
|快速|NlgN|N^2|lgN|X|pivot and partition
|归并|NlgN|NlgN|N|Y|merge sort
|基数|P(N+B)|P(N+B)|N+B|Y|count->sumcount->逆排序

## mergesort
mergesort本身比较简单，这种思路在很多题目中能够应用

In [0]:
# !! 记住这种实现，比较简洁，注意的点是: 
#  1. mid = len//2; 
#  2. if mid 一个数不需要判断，类似原地排序的 left < right 判断
#  3. 合并技巧: for i in reversed(range(len(nums))): if not right or (left and left[-1] > right[-1])
def mergeSort(nums):
    mid = (len(nums))//2
    # left < right的情况
    if mid:
        left, right = mergeSort(nums[:mid]), mergeSort(nums[mid:])
        for i in reversed(range(len(nums))):
            if not right or (left and left[-1] > right[-1]):
                nums[i] = left.pop()
            else:
                nums[i] = right.pop()
    return nums

### 例子: Ugly Number II
这个题目可以看成以mergesort，本质就是对candicate做一个mergesort（但是不要重复的），下面给了两个类似的实现，方法类似，但是方法2不用做去重的判断，而且写法更清晰

In [0]:
class Solution:
    def nthUglyNumber1(self, n):
        """
        :type n: int
        :rtype: int
        """
        # merge sort
        dp = [1]
        multIndex = [0, 0, 0] # 表示2,3,5都乘哪个dpi
        dpi = 0
        while dpi < n-1:
            # 表示2,3,5分别乘对应index的dp
            candidate = [dp[multIndex[i]]* a for i, a in enumerate([2, 3, 5])]
            minIndex, minNum = min(enumerate(candidate), key=lambda x: x[1])
            multIndex[minIndex] += 1
            if dp[-1] != minNum:
                dpi+=1
                dp.append(minNum)
        return dp[-1]
    
    def nthUglyNumber2(self, n):
        ugly = [1]
        i2, i3, i5 = 0, 0, 0
        while n > 1:
            u2, u3, u5 = 2 * ugly[i2], 3 * ugly[i3], 5 * ugly[i5]
            umin = min((u2, u3, u5))
            if umin == u2:
                i2 += 1
            if umin == u3:
                i3 += 1
            if umin == u5:
                i5 += 1
            ugly.append(umin)
            n -= 1
        return ugly[-1]

### 例子: Count of Smaller Numbers After Self

这个问题可以看成求排序的时候，把大的往右排，中间一种跳过了几个小的

In [0]:
def countSmaller(self, nums):
    """
    :type nums: List[int]
    :rtype: List[int]
    """
    # 这里nums 是 numswithindex
    def mergesort(nums, jumps):
        mid = len(nums)//2
        if mid:
            left, right = mergesort(nums[:mid], jumps),  mergesort(nums[mid:], jumps)
            for i in reversed(range(len(nums))):
                if not right or left and left[-1][1] > right[-1][1]:
                    jumps[left[-1][0]] += len(right)
                    nums[i] = left.pop()
                else:
                    nums[i] = right.pop()

        return nums

    jumps = [0] * len(nums)
    numswithindex = list(enumerate(nums))
    mergesort(numswithindex, jumps)
    return jumps

### 例子：Count of Range Sum

https://leetcode.com/problems/count-of-range-sum/description/

In [0]:
class Solution:
    def countRangeSum(self, nums, lower, upper):
        def mergesort(nums, count):
            mid = len(nums) // 2
            if mid:
                left, right = mergesort(nums[:mid], count), mergesort(nums[mid:], count)
                # left, right都是有序的了

                m, n = 0, 0
                for i, a in enumerate(left):
                    while m < len(right) and right[m] - a < lower:
                        m += 1
                    while n < len(right) and right[n] - a  <= upper:
                        n += 1
                    count[0] += n-m
                for i in reversed(range(len(nums))):
                    if not right or left and left[-1] > right[-1]:
                        nums[i] = left.pop()
                    else:
                        nums[i] = right.pop()
            return nums
        c = [0] 
        n = len(nums)
        s = [0]*(n+1)
        for i, a in enumerate(nums):
            s[i+1] = s[i] + a 
        mergesort(s, c)
        return c[0]

### 例子：Reverse Pairs
https://leetcode.com/problems/reverse-pairs/description/

In [0]:
import bisect
class Solution:
    def reversePairs(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        # use mergesort
        def mergesort(nums, c):
            mid = (0 + len(nums))//2
            if mid:
                left, right = mergesort(nums[:mid], c), mergesort(nums[mid:], c)
                l, r = 0, 0
                while l < len(left):
                    if r < len(right) and left[l] > 2*right[r]:
                        r += 1
                    else:
                        l += 1
                        c[0] += r
                #  这里没必要merge的同时count，分开count和merge反而时间复杂度更低
                #  这里使用sort而不是自己merge是因为系统库的sort运行效率更高（不考虑时间负载度）
                nums = sorted(left+right)
                # for i in reversed(range(len(nums))):
                #     if not right or (left and left[-1] >= right[-1]):
                #         nums[i] = left.pop()
                #     else:
                #         nums[i] = right.pop()
            return nums
        c = [0]
        mergesort(nums, c)
        return c[0]
                

## Radix Sort
实现见下面的例子

### 例子：maximum-gap
https://leetcode.com/problems/maximum-gap/description/
这是Radix Sort的一个应用，用Radix Sort可以在O(n)求解

注意记住下面算法里面的Radix Sort的两个实现

In [0]:
from functools import reduce
class Solution(object):
    def maximumGap(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        # Radix Sort
        # 使用嵌套array的解法, bucket解法
        def radixSort(nums):
            maxnlen = max(nums)
            index = 1
            while index <= maxnlen:
                bucket = [[] for _ in range(10)]
                for n in nums:
                    bucket[(n //index) % 10].append(n)
                nums = [n for sublist in bucket for n in sublist]
                index *= 10
            return nums

        
        # 不使用嵌套array的解法
        # https://www.cs.usfca.edu/~galles/visualization/RadixSort.html
        # 1. count; 2. sum_count; 3. order nums use sum_count (反向遍历nums) 
        #                            for n in nums[::-1]: tmp[count[(n/index)%10]--] = n
        def radixSort2(nums):
            maxnlen = max(nums)
            index = 1
            while index <= maxnlen:
                count = [0] * 10
                #1. count
                for n in nums:
                    count[(n //index) % 10] += 1

                #2. sumcount 
                for i in range(1, len(count)):
                    count[i] += count[i-1]

                #3. order num use sumcount, 注意反向遍历nums
                tmp = [0]*len(nums)
                for n in nums[::-1]:
                    count[(n //index) % 10] -=1
                    tmp[count[(n //index) % 10]] = n
                nums = tmp
                index *=10
                print(nums)
            return nums
                    
        
        if len(nums) <=1:
            return 0
        nums = radixSort2(nums)
        return max([abs(j-i) for i, j in zip(nums[:-1], nums[1:])])

### 例子：Contains Duplicate III

https://leetcode.com/problems/contains-duplicate-iii/description/

这个例子不用排序，但是利用bucket的思想可以把算法从O(nk)下降到O(n)

In [0]:
class Solution(object):
    def containsNearbyAlmostDuplicate(self, nums, k, t):
        """
        :type nums: List[int]
        :type k: int
        :type t: int
        :rtype: bool
        """
        if t < 0: return False
        # 对k取bucket那么满足条件可能在同一个bucket或者相邻的bucket
        d = {}
        w = t+1
        for i, a in enumerate(nums):
            bucketindex = a//w
            # 同一个bucke
            if bucketindex in d:
                return True
            # 相邻的bucket, 并且发现differens确实<=t
            if bucketindex - 1 in d and abs(a - d[bucketindex - 1]) <w:
                return True
            if bucketindex + 1 in d and abs(a - d[bucketindex + 1]) <w:
                return True
            
            # save this in bucket 这时候之前的bucket里面肯定没有数字，如果有，上面就返回True了
            d[bucketindex] = a
            
            # 删除距离太远的bucket，这个不会删除最近加入的，因为如果有，上面就返回True了
            if i>=k:
                del d[nums[i-k]//w]
                
        return False
            

## 快速排序
partition算法很重要，在quick select中也可以使用

In [0]:
def partition(a, left, right, pivotIndex):
     pivotValue = a[pivotIndex]
     a[pivotIndex], a[right] = a[right], a[pivotIndex] # 把 pivot 移到結尾
     storeIndex = left
     for i in range(left,right):
         if a[i] < pivotValue:
             a[storeIndex], a[i] = a[i], a[storeIndex]
             storeIndex += 1
     a[right], a[storeIndex] = a[storeIndex], a[right] # 把 pivot 移到它最後的地方
     return storeIndex # 返回 pivot 的最终位置

def quicksort(a, left, right):
    if right > left:
        #select a pivot value a[pivotIndex]
        pivotIndex = right
        pivotNewIndex = partition(a, left, right, pivotIndex)
        quicksort(a, left, pivotNewIndex-1)
        quicksort(a, pivotNewIndex+1, right)

### 例子: Kth Largest Element in an Array

https://leetcode.com/problems/kth-largest-element-in-an-array/description/

In [0]:
class Solution(object):
    def findKthSmallest(self, nums, k):
        if nums:
            pos = self.partition(nums, 0, len(nums)-1)
            if k > pos+1:
                return self.findKthSmallest(nums[pos+1:], k-pos-1)
            elif k < pos+1:
                return self.findKthSmallest(nums[:pos], k)
            else:
                return nums[pos]

    # choose the right-most element as pivot   
    def partition(self, nums, l, r):
        pivotvalue = nums[r]
        index = l
        while l <= r:
            if nums[l] < pivotvalue:
                nums[l], nums[index] = nums[index], nums[l]
                index += 1
            l += 1
        # pivot to last index
        nums[r], nums[index] = nums[index], nums[r]
        return index
    
    
    def findKthLargest(self, nums, k):
        return self.findKthSmallest(nums, len(nums) - k +1 )
    

# 树的遍历和应用


## 树的遍历
二叉树的广度优先遍历和树的深度优先(前序/中序/后序)遍历不太一样，前/中/后序遍历使用递归，也就是栈的思想对二叉树进行遍历，
广度优先一般使用队列的思想对二叉树进行遍历。

- 深度优先：先访问子节点，再访问父节点，最后访问第二个子节点。根据根节点相对于左右子节点的访问先后顺序又可细分为以下三种方式。前+中 或者 后+中 可以唯一确定一个树
  - 前序(pre-order)：**先根后左再右**
  - 中序(in-order)：**先左后根再右**
  - 后序(post-order)：**先左后右再根**
- 广度优先：先访问根节点，沿着树的宽度遍历子节点，直到所有节点均被访问为止。


### 递归
递归解法直接根据概念即可

In [0]:
class Traversal(object):

    def visit(self, node):
        print(node.val, end=" ")

    # dfs
    def preorder(self, root):
        if root:
            self.visit(root)
            self.preorder(root.left)
            self.preorder(root.right)

    def inorder(self, root):
        if root:
            self.inorder(root.left)
            self.visit(root)
            self.inorder(root.right)

    def postorder(self, root):
        if root:
            self.postorder(root.left)
            self.postorder(root.right)
            self.visit(root)

In [0]:
def buildTestTree():
  #         7
  #      /     \
  #    5        6
  #  /   \     /   \
  # 1     2   3     4                        

  node1 = TreeNode(1)
  node2 = TreeNode(2)
  node3 = TreeNode(3)
  node4 = TreeNode(4)
  node5 = TreeNode(5, left = node1, right= node2)
  node6 = TreeNode(6, left = node3, right = node4)
  node7 = TreeNode(7, left = node5, right = node6)
  return node7

root = buildTestTree()
Traversal().preorder(root); print()
Traversal().inorder(root); print()
Traversal().postorder(root); print()

### 迭代
- 前序(pre-order)：**先右后左入栈**
- 中序(in-order)：**有左就入左子树(遍历), 否则pop，pop之后右入栈**
- 后序(post-order)：**有儿子而且没有visit就右左儿子入栈，否则pop**
- bfs: **使用队列**

In [0]:
class Traversal(object):

    def visit(self, node):
        print(node.val, end=" ")

    # dfs
    def preorder(self, root):
        stack=[root]
        while stack:
          node = stack.pop()
          self.visit(node)
          if node.right:
            stack.append(node.right)
          if node.left:
            stack.append(node.left)
          

    def inorder(self, root):
        stack=[]
        while root or stack:
          if root:
            stack.append(root)
            root = root.left
          else:
            root = stack.pop()
            self.visit(root)
            root = root.right

    def postorder(self, root):
        stack = [root]
        prev = None
        while stack:
          node = stack[-1]
          haschild = node.left or node.right
          visit = prev and (prev == node.left or prev == node.right)
          if haschild and not visit:
            if node.right:
              stack.append(node.right)
            if node.left:
              stack.append(node.left)
          else:
            self.visit(stack.pop())
            prev = node
            
    def bfs(self, root):
        queue = [root]
        while queue:
            node = queue.pop(0)
            self.visit(node)
            if node.left:
                queue.append(node.left)
            if node.right:
                queue.append(node.right)

In [0]:
root = buildTestTree()
Traversal().preorder(root); print()
Traversal().inorder(root); print()
Traversal().postorder(root); print()
Traversal().bfs(root); print()

### Morris
Morris是另一种树遍历算法，它能够：
1. **O(1)空间复杂度**，使用常数空间；
2. 二叉树的形状不被破坏。

具体做法为（三个基本相同，只是输出方式有一点点不同）：
- 前序(pre-order)：**无左则输出走右，有左则找中序前驱，前驱无右儿子则连线输出走左，前驱有右儿子则断线走右**
- 中序(in-order)：**无左则输出走右，有左则找中序前驱，前驱无右儿子则连线走左，前驱有右儿子则断线输出走右**
- 后序(post-order)：**无左则走右，有左则找中序前驱，前驱无右儿子则连线走左，前驱有右儿子则断线  倒序输出从当前节点的左孩子到该前驱节点(包括)这条路径上的所有节点 走右**

In [0]:
class Traversal(object):

    def visit(self, node):
        print(node.val, end=" ")

    # dfs
    def preorder(self, root):
        cur, prev = root, None
        while cur:
          if not cur.left:
            # 无左则输出走右
            self.visit(cur)
            cur = cur.right
          else:
            # 找中序前驱
            prev = cur.left
            while prev.right!= None and prev.right != cur:
              prev = prev.right
            if not prev.right:
              # 前驱无右儿子则连线输出走左
              prev.right = cur
              self.visit(cur)
              cur = cur.left
            else:
              # 前驱有右儿子则断线走右
              prev.right = None
              cur = cur.right
          

    def inorder(self, root):
        cur, prev = root, None
        while cur:
          if not cur.left:
            # 无左则输出走右
            self.visit(cur)
            cur = cur.right
          else:
            # 找中序前驱
            prev = cur.left
            while prev.right!= None and prev.right != cur:
              prev = prev.right
            if not prev.right:
              # 前驱无右儿子则连线走左
              prev.right = cur
              cur = cur.left
            else:
              # 前驱有右儿子则断线输出走右
              prev.right = None
              self.visit(cur)
              cur = cur.right
             

    def postorder(self, root):
        def reversevist(start, end):
          s = [str(end.val)]
          while start and start != end:
            s.append(str(start.val))
            start = start.right
          print(" ".join(s[::-1]), end=" ")
        
        dummy, prev = TreeNode(0), None
        dummy.left = root
        cur = dummy
        while cur:
          if not cur.left:
            # 无左则走右
            cur = cur.right
          else:
            # 找中序前驱
            prev = cur.left
            while prev.right!= None and prev.right != cur:
              prev = prev.right
            if not prev.right:
              # 前驱无右儿子则连线走左
              prev.right = cur
              cur = cur.left
            else:
              # 前驱有右儿子则断线  倒序输出从当前节点的左孩子到该前驱节点这条路径上的所有节点 走右
              reversevist(cur.left, prev)
              prev.right = None
              cur = cur.right
            

In [0]:
root = buildTestTree()
Traversal().preorder(root); print()
Traversal().inorder(root); print()
Traversal().postorder(root); print()

### 其他有趣的遍历算法
不用队列来进行层序遍历
下面这个例子使用来递归，等价于使用了栈

In [0]:
class Traversal(object):

    def visit(self, node):
        print(node.val, end=" ")

    def bfs(self, root, levels):
      def visitlevel(node, target_level, current_level):
        if target_level > current_level:
          if node.left:
            visitlevel(node.left, target_level, current_level+1)
          if node.right:
            visitlevel(node.right,target_level, current_level+1)
        else:
          self.visit(node)
       
      for i in range(levels):
        visitlevel(root, i, 0)

In [0]:
root = buildTestTree()
Traversal().bfs(root, 3); print()

## 应用
一个问题能抽象成树的遍历问题之后，实现就比较简单了, 比如下面的word search, 或者一些subset、 combination也常能看成树的遍历问题

### 例子：Word Search
https://leetcode.com/problems/word-search/description/

这个题目用bfs会发现很困难，用bfs加递归、回溯的方式比较简单

In [0]:
class Solution:
    def exist(self, board, word):
        """
        :type board: List[List[str]]
        :type word: str
        :rtype: bool
        """
        # dfs
        def dfs(i, j, board, word): #i, j is current index
          if len(word)==0:
            return True
          if i<0 or j<0 or i>=len(board) or j>=len(board[0]) or board[i][j] != word[0]:
            return False
          tmp=board[i][j] 
          board[i][j]="#" #用于回溯
          ret=dfs(i-1, j, board, word[1:]) or dfs(i+1, j, board, word[1:]) or dfs(i, j-1, board, word[1:]) or dfs(i, j+1, board, word[1:])
          board[i][j]=tmp #回溯
          return ret
          
        for i in range(len(board)):
          for j in range(len(board[0])):
            if dfs(i, j, board, word):
              return True
        return False
                

In [0]:
print(Solution().exist([["A","B","C","E"],["S","F","C","S"],["A","D","E","E"]], "ABCCED"))
print(Solution().exist([["C","A","A"],["A","A","A"],["B","C","D"]], "AAB"))
print(Solution().exist([["A","B","C","E"],["S","F","E","S"],["A","D","E","E"]], "ABCESEEEFS"))

### 例子：中序遍历的应用举例
https://leetcode.com/problems/validate-binary-search-tree/discuss/32112/Learn-one-iterative-inorder-traversal-apply-it-to-multiple-tree-questions-(Java-Solution)

## 例子：Remove Invalid Parentheses

https://leetcode.com/problems/remove-invalid-parentheses/description/

这道题只能做删除操作，那么一种最简单的办法就是暴力搜索（依次删除递归）看结果，这种办法也可以看成是BFS，按层不断接近最后的结果。BFS可以用queue来解决，这里使用filter简化了代码。

进一步的，代码中没有考虑BFS中的很多可以剪枝的情况，比如((), 删第一个和第二个是一样的，那么可以剪掉一个树枝。

In [0]:
class Solution(object):
    def removeInvalidParentheses(self, s):
        """
        :type s: str
        :rtype: List[str]
        """
        def isvalid(s):
            d = {"(" : 1, ")": -1}
            cl = 0
            for a in s:
                if a in d:
                    cl += d[a]
                    if cl < 0:
                        return False
            return cl == 0
        
        # use set to avoid duplicate
        level = {s}
        while True:
            valid = filter(isvalid, level)
            if valid:
                return valid
            level = {s[:i] + s[i+1:] for i in range(len(s)) for s in level}

## 例子：Path Sum  III

https://leetcode.com/problems/path-sum-iii/description/

遍历的时候确定分割成几个条件能简化问题，比如这题里面dfs就有两种情况node取，或者node不取。node如果比较简单，递归原函数就可以，但是node取，需要另外定义一个函数出来才能描述这种情况。

In [0]:
class Solution:
    def pathSum(self, root, sum):
        """
        :type root: TreeNode
        :type sum: int
        :rtype: int
        """
        # 必须包含
        def pathSumFrom(node, sum):
            if not node:
                return 0
            a = 1 if node.val == sum else 0
            return a + pathSumFrom(node.left, sum- node.val) + pathSumFrom(node.right, sum- node.val)
        
        if not root:
            return 0
        return pathSumFrom(root, sum) + self.pathSum(root.left, sum) + self.pathSum(root.right, sum)

## 例子：Subtree of Another Tree
https://leetcode.com/problems/subtree-of-another-tree/description/
思路是把树转化成string，但是如果单纯遍历并不能表达“是subtree”这种情况，所以把一些其他字符也加进去用于表达是node等信息

In [0]:
class Solution:
    def isSubtree(self, s, t):
        """
        :type s: TreeNode
        :type t: TreeNode
        :rtype: bool
        """
        def convert(p):
            return "^" + str(p.val) + "#" + convert(p.left) + convert(p.right) if p else "$"
        
        #print(convert(t), convert(s))
        return convert(t) in convert(s)

## 例子：Sum of Distances in Tree
https://leetcode.com/problems/sum-of-distances-in-tree/description/

分析 https://leetcode.com/problems/sum-of-distances-in-tree/discuss/130567/Two-traversals-O(N)-python-solution-with-Explanation

遍历第一遍 算count, 同时把当前res[root]算出来
遍历第二遍 根据count和 res[root] 把 res[每个i]都算出来

In [0]:
import collections
class Solution:
    def sumOfDistancesInTree(self, N, edges):
        """
        :type N: int
        :type edges: List[List[int]]
        :rtype: List[int]
        """
        # 树，记录每个node的儿子
        tree = collections.defaultdict(set)
        # 记录结果
        res = [0] * N
        # 记录每个node的儿子个数
        count = [0] * N 
        
        for s, e in edges:
            tree[s].add(e)
            tree[e].add(s)

        # post order, 其实每个节点都可以是root，第一遍取0
        def post_order(root=0, seen=set()):
            seen.add(root)
            for i in tree[root]:
                if i not in seen:
                    post_order(i, seen)
                    count[root] += count[i]
                    # 对于树，每往上走一层，所有儿子的距离都要+1， 所以增加之后为 res[i] + count[i]
                    res[root] += res[i] + count[i]
            count[root] += 1 # count包括自己 +1
        
        # 前序遍历, 从root到儿子，root的res已经算出来了，根据count 可以算出儿子的res
        def pre_order(root=0, seen=set()):
            seen.add(root)
            for i in tree[root]:
                if i not in seen:
                    #seen.add(i)
                    res[i] = res[root] - count[i] + N - count[i] #(i这边相对于root减少count[i], 另一边相对于root增加N-count[i])
                    pre_order(i, seen)
                    
        post_order()
        pre_order()
        return res        

### 例子：Shortest Path Visiting All Nodes

https://leetcode.com/problems/shortest-path-visiting-all-nodes/description/
很重要的一个思路: 

起点不是一个点 去**扩张**，而是起点是所有边界，终点是一个点，bfs是**收缩**

**用二进制数字表示state也比较常见**


In [0]:
import collections
class Solution:
    def shortestPathLength(self, graph):
        n = len(graph)
        final = (1 << n) - 1 # 用11111表示每个node都访问过了
        steps = 0
        seen, q = set(), collections.deque([(i, 0, 1 << i) for i in range(len(graph))])
        # 使用bfs, 起点是00001, 00010, 00100, 01000, 10000 终点是11111
        # 这个bfs的思路也很常见，起点不是一个点 去**扩张**，而是起点是所有边界，终点是一个点，bfs是**收缩**
        while q:
            node, steps, state = q.popleft()
            if state == final: 
                return steps
            for v in graph[node]:
                if (state | 1 << v, v) not in seen:
                    q.append((v, steps + 1, state | 1 << v))
                    seen.add((state | 1 << v, v))
                    
        return steps

# 数字和数学

## 有用的数字技巧

1. num&(num-1) == 0可以用来判断数字是2的幂
2. 一个有用的公式如果 (x # y) @ y = x 那么

```python
def swap(x, y):
  x = x # y
  y = x @ y
  x = x @ y

比如这里的 #, @ = '+', '-' ; 或者 #,@ = 'xor', 'xor' 等等都是有效的 (异或:不一样则为1,一样为0; a xor a = 0, 0 xor a = a)
```
3. 保留一个二进制的最后一位1，其他都置0 => a- (a & (a-1)); 比如1010 => 1010 - 1010 & 1001 = 0010 (注意括号)
4. 保留一个二进制的最后一位1的另一种做法 a & (-a), 例如1010 -》 0010，这个算法在binary index tree里面用到了
5. 出现几次的问题
  1. 只有一个出现1次，其他都出现2次 => 异或
  2. 只有一个出现1次，其他都出现3次 => 按位求和求3的余
  3. 只有两个出现1次，其他都出现2次 => 异或 -> 分组 -> 异或
  4. 只有1个出现多次，其他都出现1次（n+1个数字都<n） => 看成是link list有环，求环入口的问题 https://leetcode.com/problems/find-the-duplicate-number/description/
  5. 一个数出现多于n/2次，或者两个数出>= n/3次：Boyer-Moore Majority Vote

## 质数/素数的应用
### Group Anagrams
https://leetcode.com/problems/group-anagrams/description/


## 排列组合
https://leetcode.com/problems/unique-paths/description/

这个可以看成robot要走m+n-2步，里面选n-1步往下走，即C(m+n-2, n-1)

而有obstacles的情况很难用组合来抽象

https://leetcode.com/problems/unique-paths-ii/description/

可以用dp来解， dp[i][j] 表示从0,0 到 i, j 的path数量
那么dp[i][j] = dp[i-1][j] + dp[i][j-1] if (i, j not obstacle) else 0

In [0]:
class Solution(object):
    def uniquePaths(self, m, n):
        """
        :type m: int
        :type n: int
        :rtype: int
        """
        def nCr(n,r):
            f = math.factorial
            return f(n) // f(r) // f(n-r)
        return nCr(m+n-2, n-1)
      
    def uniquePathsWithObstacles(self):
      # 略
      pass
        

## 经典：找因子，质因子，公共因子


```python
# 1. 找因子，从1，遍历到 math.sqrt(a), 加上除数和商
def find_factor(a):
    ret = []
    for i in range(1, int(math.sqrt(a)) + 1):
        if a%i == 0:
            m = a//i
            # 这里保持顺序只是为了好看
            bisect.insort(ret, i)
            if m != i:
                bisect.insort(ret, m)
    return ret
  
# 2. 找质因子:Pollard Rho因数分解 经典算法
def prime_factor(n):
    ret = []
    i = 2
    while i**2 <= n:
        while n%i==0:
            ret.append(i)
            n //= i
        i += 1
    if n> 1:
        ret.append(n);
    return ret
  
# 3. 求公因子
# 两个数的公因子包含必定是两个数的最大公约数的因子
def gcd(a, b):
    # 辗转相除法求最大公约数
    return a if b == 0 else gcd(b, a%b)

def common_factors(a, b):
  return find_factor(gcd(a, b))
```

## 例子：Single Number II
https://leetcode.com/problems/single-number-ii/description/

出现一次，异或求解
出现n次，可以利用这样的性质：每一位上的数字都加一起来，那么%n就是一个只出现一次的那个数字在这一位的数字

## 例子：Bitwise AND of Numbers Range
https://leetcode.com/problems/bitwise-and-of-numbers-range/description/

## 例子：Counting Bits
https://leetcode.com/problems/counting-bits/description/
常见的思路，处理位数有关的问题的时候考虑 f(i) 和 f(i<<1) 或者f(i>>1)的关系

l[i] = l[i>>1] + (i&1)

## 例子：Super Pow
https://leetcode.com/problems/super-pow/description/
这个例子考察两个东西：

1. (a*b)%c = (a%c)*(b%c)%c
2. divide and conquer的的方式计算pow

In [0]:
class Solution(object):
    def superPow(self, a, b):
        """
        :type a: int
        :type b: List[int]
        :rtype: int
        """
        
        # 二分法计算pow
        def pow(x, n):
            if n == 0:
                return 1
            elif n == 1:
                return x % 1337
            else:
                return pow(x, n//2) * pow(x, n - n//2) % 1337
        
        dp = {}
        def pow_with_dp(x, n):
            if (x, n) in dp:
                return dp[(x, n)]
            if n == 0:
                return 1
            elif n == 1:
                return x % 1337
            else:
                ret = pow(x, n//2) * pow(x, n - n//2) % 1337
                dp[(x, n)] = ret
                return ret


        # 利用 (a*b)%c = (a%c*b%c) %c
        res = 1
        for n in b:
            res = pow_with_dp(res, 10) * pow_with_dp(a, n) % 1337
        return res

## 例子：Minimum Moves to Equal Array Elements

https://leetcode.com/problems/minimum-moves-to-equal-array-elements/description/

这道题关键之处在于找到**等价表示**，对n-1个数加一，和对一个数减1的效果是一样的（在变成相等的步数上），转化成对一个数减1。思路就简单很多了。

## 例子：Minimum Moves to Equal Array Elements II

```
中位数性质:
median: min(    sum(abs(median-x))     )
证明: 将样本值从小到大排列，S(a)＝|x(1)-a|+|X(2)-a|+...+|x(n)-a|,考虑a的位置。
     如果在a左面和右面的观测值个数不同，总可以找到另一个a，使得S(a)更小。      所以只有当a左面和右面的观测值个数相同时，S(a)才达到最小值。
```

## 例子：Sum of Square Numbers
```
Input: 5
Output: True
Explanation: 1 * 1 + 2 * 2 = 5
```
重要的遍历技巧，如果用连个范围去遍历复杂度过高，遍历一个范围，求出另一个值，看这个值是否满足条件会时间复杂度降低很多。

In [0]:
import math
class Solution:
    def judgeSquareSum(self, c):
        """
        :type c: int
        :rtype: bool
        """
        def is_square(a):
            return int(math.sqrt(a))**2 == a
        
        d = int(math.sqrt(c))
        for b in range(d//2, d+1):
            if is_square(c - b**2):
                return True
        return False

## 例子：Factorial Trailing Zeroes 和 Preimage Size of Factorial Zeroes Function

给定一个数，如何找这个数的阶乘末尾的0的个数，本质是找因子里面有多少5，那么 5！里面有一个5， 10！里面有两个5，到目前为止都是n//5个，但是 25！会比前面的增加两个5，可以推论出下面的递推公式：

```python
#递归写法
#return 0 if n == 0 else n / 5 + self.trailingZeroes(n / 5)
class Solution:
    def trailingZeroes(self, n):
        """
        :type n: int
        :rtype: int
        """
        #循环写法
        ret = 0
        while n >= 5:
            ret += n//5
            n = n//5
        return ret
```

看总结：https://leetcode.com/problems/preimage-size-of-factorial-zeroes-function/discuss/117821/Four-binary-search-solutions-based-on-different-ideas

# 动态规划

##  背包问题
### 01背包问题
有n个物品，价值v = [v1,v2,v3…]，体积c = [c1,c2,c3…]，放入总容量为 totalCapacity 的背包中，求能获得的最大价值是多少？

01背包问题是典型的动态规划问题，他的变种却往往不容易想到动态规划的一个主要原因是：迭代中把**总容量**作为一个迭代项，这点可能有违一般的迭代思路。

```python
# f[i][j]表示取到物品i, 而且总容量为j时候的最优解
f[i][j] = max(f[i-1][j],f[i-1][j-c[i]] + v[i])
```

### 01背包的变种
#### 1. 不考虑价值，刚好装满
这个比较简单 f[i][j] 做点变化即可

#### 2. 不考虑价值，希望最多物品（假设空间无空隙）
这个也叫最优装载问题
这个可以用贪心法在O(N)时间解决：优先选c最小的
证明：假设x1...xn已经排序，最优解已经包含了x1，那么剩下的空间如果还能放下xk, 必然也能放下x2, 所以xk -> xk+1，归纳可知为最优解

### 完全背包问题
递归公式只是有01背包少许不同

```python
# f[i][j]表示取到物品i, 而且总容量为j时候的最优解
f[i][j] = max(f[i-1][j],f[i][j-c[i]] + v[i])
```

完全背包问题可以限制totalCapacity，或者如果totalCapacity限制的同时还限制了n个商品最多只能有n[i]个怎么办，第i个商品最多只能用min(n[i], totalCapacity/ci)个

#### 完全背包问题二进制优化
用这种办法可以把完全背包问题转换成01背包问题，运算速度加快
原理是：**一个正整数n可以被分解成1,2,4,…,2^(k-1),n-(2^k+1)(即n-前面数的和), k是满足n>2^k+1的最大整数，且1～n之内的所有整数均可以唯一表示成这些数中某几个数的和的形式。**

那么完全背包问题可以看成01背包问题，比如c=[2, 6], totalCapacity = 20, 那么物品1可以选择最多10件，c1可以看成选 1, 2, 4, 3  * c1 四种商品，因为不管选几件，都可以用这四种新商品表示出来，同理对6可以选择1， 2件。那么问题就变成了c=[2,4,6,8,6,12] 这几种商品的01背包问题。


### 例子：Partition Equal Subset Sum
https://leetcode.com/problems/partition-equal-subset-sum/description/
这是一个01 背包问题

In [0]:
class Solution:
    def canPartition(self, nums):
        """
        :type nums: List[int]
        :rtype: bool
        """
        sumn = sum(nums)
        if sumn%2 != 0:
            return False
        target = sumn //2
        dp = [False] * (target+1)
        dp[0] = True
        for i in range(len(nums)):
            for j in reversed(range(target+1)):
                if nums[i] <= j:
                    #这里是把二维矩阵改造成一维的思路
                    #dp[i][j] = dp[i-1][j] or dp[i][j-nums[i]]
                    dp[j] = dp[j] or dp[j-nums[i]]
                #else:
                    #dp[i][j] = dp[i-1][j]
                    #dp[j] = dp[j]
        
        return dp[-1]

### 例子：Ones and Zeroes
https://leetcode.com/problems/ones-and-zeroes/description/

In [0]:
class Solution(object):
    def findMaxForm(self, strs, m, n):
        """
        :type strs: List[str]
        :type m: int
        :type n: int
        :rtype: int
        """
        # 这是一个01背包问题，对于一个s 可以选择放或者不放 
        # dp[i][j][k] = max(dp[i][j][k-1], dp[i-x][j-y][k] + 1) for (x, y) in strs
        # 使用bottom up 遍历, (为什么使用reversed?) 可以省略掉k这个维度
        dp = [[0] * (n+1) for _ in range(m+1)]
        for s in strs:
            x, y = s.count("0"), s.count("1")
            for i in reversed(range(x, m+1)):
                for j in reversed(range(y, n+1)):
                    dp[i][j] = max(dp[i][j], dp[i-x][j-y] + 1)
        return dp[-1][-1]
                        

## 例子：Longest Palindromic Substring
动态规划常用矩阵保存遍历状态

https://leetcode.com/problems/longest-palindromic-substring/description/

dp在这里不是最优算法，但是对于学习dp算法，这个例子很好

这里我们假设dp[i][j] 表示 s[i, j] 为回文, value为这个回文的长度 (如果不是回文那么填0)
那么：
1.  dp[i][j] = dp[i+1][j-1] + 1 if s[i] == s[j ] else 0
2.  dp[i][i] = 1
3.  dp[i][i+1] = 2 if s[i] == s[i+1] else 0


另一种dp表示办法
dp[i] 表示以 s[i] 结尾的最大回文长度, 那么:

```
dp[i] = max(dp[i-1]+2 if s[i]==s[i-d[i-1]-1]
                    , 2 if s[i] == s[i-1]
                    , 1 )
```


 这个题目也有O(n) 算法 https://www.felix021.com/blog/read.php?2040

In [0]:
class Solution:
    def longestPalindrome(self, s):
        """
        :type s: str
        :rtype: str
        """
        a = len(s)
        dp = [[0] * a for _ in range(a)]
        maxl = 0
        left, right = 0, 0 
        # 基准情况: 长度为 1, 2的情况
        for i in range(len(s)):
          dp[i][i] = 1
          if i+1 < len(s) and s[i+1] == s[i]:
            dp[i][i+1] = 2
            left, right = i, i+1
            
          
        # 长度为l， l >= 3的情况
        for l in range(3, a+1):
          for i in range(len(s)-l+1):
            j = i+l-1
            if s[i] == s[j] and dp[i+1][j-1] != 0:
              dp[i][j] = dp[i+1][j-1] + 2
              left, right = i, j
        #print(dp)
        return s[left:right+1]
                      
          
           

In [0]:
print(Solution().longestPalindrome("ccc"))

## 例子：Regular Expression Matching
https://leetcode.com/problems/regular-expression-matching/description/

Given an input string (s) and a pattern (p), implement regular expression matching with support for '.' and '*'.

dp[i][j] 表示 p[:i] 匹配 s[:j]

 1.  if p[i-1] != "*" : dp[i][j] = dp[i - 1][j - 1] && s[j - 1] == p[i - 1]
 2.  if p[i-1] == "*": 假设 p[i-2] 为 x，那么我们现在假设
  1.  x* match了0次，那么dp[i][j] = dp[i - 2][j]
  2.  x* match了>=1次，那么dp[i][j] = dp[i][j-1] && x == s[j-1]
 
注意上面写的"==" 需要考虑 p为'.' 的情况



In [0]:
class Solution:
    def isMatch(self, s, p):
        """
        :type s: str
        :type p: str
        :rtype: bool
        """
        def singleMatch(s1, p1):
          return s1 == p1 or p1 == "."
        
        dp = [[False] * (len(s)+1) for _ in range(len(p)+1)]
        dp[0][0] = True
        # 边界情况, s为空，p不为空
        for i in range(2, len(p) + 1):
          dp[i][0] = dp[i-2][0] and p[i-1] == "*"
        for i in range(1, len(p) + 1):
          for j in range(1, len(s) + 1):
            if p[i-1] != '*':
              dp[i][j] = dp[i - 1][j - 1] and singleMatch(s[j - 1], p[i - 1])
            else:
              dp[i][j] = dp[i-2][j] or \
                        (dp[i][j-1] and singleMatch(s[j-1], p[i-2]))
              
        return dp[-1][-1]
        
        

In [0]:
print(Solution().isMatch("aab", "a*b"))
print(Solution().isMatch("baa", "ba*"))
print(Solution().isMatch("baa", "b.*"))

## 例子：Jump Game
https://leetcode.com/problems/jump-game/description/
可以在线性时间内求解，类似一种greed方法，不断扩展可以reach的区域

## 例子：Jump Game II
https://leetcode.com/problems/jump-game-ii/description/

这个用dp只是提一个思路，实际上dp算法在leetcode会超时，因为这是一个最坏O(n^2)的算法，**而使用dfs 只要O(n)**

In [0]:
class Solution(object):
    def jumpDP(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        if len(nums) < 2:
            return 0
        dp = [0] * len(nums)
        for i in reversed(range(0, len(nums) - 1)):
            n = nums[i]
            if n > 0:
                dp[i] = min([dp[i + j] + 1 if i + j < len(dp) else 1 for j in range(1, n + 1)])
            else:
                dp[i] = 999999

        return dp[0]   
     
    def jumpDFS(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        n, start, end, step = len(nums), 0, 0, 0 
        # start, end 表示当前遍历的这一层的开始、结束位置
        #  比如对于23114遍历的树为
        #     2 || 3 1 || 1 1 4
        #  对于 2 3 1 0 1 2 1 
        #      2 || 3 1 || 0 1 || 2 || 1 
        while end < n -1:
          step += 1
          maxend = end + 1
          for i in range(start, end+1):
            if i + nums[i] >= n - 1:
              return step
            maxend = max(maxend, i + nums[i])
          start, end = end + 1, maxend
        
        return step 
      
      
                

In [0]:
print(Solution().jumpDP([2,3,1,1,4]))
print(Solution().jumpDFS([2,3,1,1,4]))
print(Solution().jumpDFS([2,3,1, 0, 1,2,1]))

## 例子：Edit Distance
https://leetcode.com/problems/edit-distance/description/
假设dp[i][j] 表示把单词s1 变成 s2的距离。
对一个单词w1（s1[:i]）, 要把他变成w2 (s1[:j]), 假设w1末尾是c，w2末尾是d，那么不管w1和w2的长度，都有：
- 如果c==d, dp[i][j] == dp[i--1][j--1]
- 如果c!=d, 那么可以用d替换c：dp[i][j] == dp[i--1][j--1] + 1
-                  可以加上d:  dp[i][j] == dp[i][j--1] + 1
-                  可以删除c:  dp[i][j] == dp[i--1][j] + 1

```python
// 考虑处理末尾的情况，要变一致，末尾肯定是要处理的
// case 1
-----c
 ||              
------d

// case 2
-----c
  ||               
----d

// case 3
-----c
  ||            
-----d
```


##例子： Decode Ways
dp技巧，从后往前有时候会方便很多
https://leetcode.com/problems/decode-ways/description/

## 例子： Interleaving String

https://leetcode.com/problems/interleaving-string/description/

这个题除了dp还给了一个有趣的bfs解法


Say s1 = "aab" and s2 = "abc". s3 = "aaabcb". Then the board looks like, 看成一个寻路问题，从左上角走到右下角，可以向右或者向下
```
o--a--o--b--o--c--o
|     |     |     |
a     a     a     a
|     |     |     |
o--a--o--b--o--c--o
|     |     |     |
a     a     a     a
|     |     |     |
o--a--o--b--o--c--o
|     |     |     |
b     b     b     b
|     |     |     |
o--a--o--b--o--c--o
```

In [0]:
class Solution:
    def isInterleave(self, s1, s2, s3):
        """
        :type s1: str
        :type s2: str
        :type s3: str
        :rtype: bool
        """
        if len(s3) != (len(s1) + len(s2)):
            return False
        #dp[i][j] 表示s1[:i], 和s2[:j] 可以isInterleave变成s3[:i+j]
        dp = [[False]* (len(s2)+1) for _ in range(len(s1) + 1)]
        for i in range(len(s1)+1):
            for j in range(len(s2)+1):
                if i ==0 and j == 0:
                    dp[i][j] = True
                else:
                    dp[i][j] = (i>=1 and dp[i-1][j] and s1[i-1] == s3[i+j-1]) \
                                or (j >=1 and dp[i][j-1] and s2[j-1] == s3[i+j-1])
        return dp[-1][-1]
        
        

## 例子：Best Time to Buy and Sell Stock III
https://leetcode.com/problems/best-time-to-buy-and-sell-stock-iii/description/

这道题可以看成一个类似背包的问题:

```
dp[i, j] 为最大i次操作，而且操作到第j个price的最大收益
那么（第j个肯定是卖操作）
dp[i,j]= max(dp[i,j], prices[i]-prices[j]+dp[k-1, j]) {j=0...i-1}

意思是卖任何一个

// 临界条件
dp[0, j] = 0;
dp[i, 0] = 0; 
```



这道题也可以这么看：假设有4次操作buy1, sell1,buy2, sell2
，几个变量分别指代几次操作后的收益（buy之后收益是负的），那么操作的目的是尽可能的便宜买贵卖，下面的贪心算法对于一次和两次操作都是有效的，其实这种算法可以看成是dp算法的特殊情况(n=2)
```
buy1 = Math.max(buy1, -prices[i]);
sell1 = Math.max(sell1, buy1 + prices[i]);
buy2 = Math.max(buy2, sell1 - prices[i]);
sell2 = Math.max(sell2, buy2 + prices[i])
```

## 例子：Best Time to Buy and Sell Stock with Cooldown

https://leetcode.com/problems/best-time-to-buy-and-sell-stock-with-cooldown/description/

## 例子：Palindrome Partitioning II
https://leetcode.com/problems/palindrome-partitioning-ii/description/

dp[i] 表示 s[:i] 最少cut 多少次那么 :

dp[i] = min ( dp[j - 1] + 1 (j <= i), if s[j, i] is palindrome )

而 s[j, i] 是palindrome也可以用dp 也解决

dps[j, i] = dps[j +1, i-1] if s[i] == s[j]

```
a   b   a   |   c  c
                j  i
       j-1  |  [j, i] is palindrome
   cut(j-1) +  1
```


In [0]:
class Solution:
    def minCut(self, s):
        dpCut = [0] * len(s)
        dpPalindrome = [[False] * len(s) for _ in range(len(s))]
        for i in range(len(s)):
            dpCut[i] = i
            for j in range(i+1):
                if s[j] == s[i] and (j+1 > i-1 or dpPalindrome[j+1][i-1]):
                    dpPalindrome[j][i] = True
                    dpCut[i] = 0 if j == 0 else min(dpCut[i], dpCut[j-1] + 1)    
        return dpCut[-1]

## 例子：DP变化- Burst Balloons and Remove Boxes
https://leetcode.com/problems/burst-balloons/description/
https://leetcode.com/problems/remove-boxes/description/

**Burst Balloons**
dp的思路是选第一个，下面列出了两种解法

这类dp问题很难找到递推关系：因为不是self-contained，问题本身依赖于问题的外部变量. In this case, I shall call that the definition of the subproblem is not self-contained and its solution relies on information external to the subproblem itself. 这时候就需要变化一些递推关系，比如Burst Balloons里面如果用T(i, j)表示i~j 的结果就不行，因为同时依赖nums[i-1] 和 nums[j+1], 变化一下：用T(i, j) 表示 i+1~j-1, 那么就变成了一个self contain的问题，这时候: 

```python
T(i, j) = max(T(i, k) + T(k, j) + nums[k-1]*nums[k]*nums[k+1] for k in range(i+1, j))
```

**Remove Boxes** 
这道题类似，直接找递推关系很困难，直接dfs是肯定会TLE的(O(n!))
T(i, j) 不仅和自己有关，还和自己外面的变量有关系
```
假设我们要找这样的递推关系:
T(i, j) = T(i+k, j) + k*k (i~k+i-1)是相同的
这个式子其实没有表达出最优解，因为这个式子表示remove i和他相邻的，但是其实这一段可以留着之后再remove可能结果更好

为了表达这种关系，我们引进了k，表示在i左边有的和i相同的颜色的盒子的个数
那么我们要求解的就是i
T(0, n-1, 0)

递推关系变成了 
T(i, j, k) = 
max:
  T(i+k, j) + (k+1)*(k+1)
  T(i+1, m-1, 0) + T(m, j, k+1) for m in (i, j) if m颜色 == i
```
**Strange Printe**

https://leetcode.com/problems/strange-printer/description/

In [0]:
class Solution:
    def maxCoins1(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        # 使用递归比较好理解, 关键是选哪个作为第一个, 这里求i -- j  !! 不包括 i, j
        def dp(i, j, nums, dic):
            # j -i = 1的情况其实就是没有 
            if (i, j) in dic or j - i == 1:
                return dic.get((i, j), 0)
            else:
                ret = max(dp(i, k, nums, dic) + dp(k, j, nums, dic) + nums[i]*nums[k]*nums[j] for k in range(i+1, j))
                dic[(i, j)] = ret
                return ret
        nums = [1] + nums + [1]
        return dp(0, len(nums)-1, nums, {})
        
        
    # 不用递归就要bottom up,这个 bottom-up的方式应该是gap从2->n 
    def maxCoins(self, nums):
        nums = [1] + nums + [1] # build the complete array 
        n = len(nums)
        dp = [[0] * n for _ in range(n)]
        for gap in range(2, n):
            for left in range(0, n - gap):
                right = left + gap
                for i in range(left + 1,right):
                    dp[left][right] = max(dp[left][right],
                       nums[left] * nums[i] * nums[right] + dp[left][i] + dp[i][right])
        return dp[0][n-1]
                    
        

In [0]:
class Solution(object):
    def removeBoxes(self, boxes):
        """
        :type boxes: List[int]
        :rtype: int
        """
        def helper(boxes, i, j, k, dp):
            if i > j:
                return 0
            if (i, j, k) in dp:
                return dp[(i, j, k)]
            
            # 前面颜色相同的，注意这里会留一个i
            while i + 1 < j and boxes[i+1] == boxes[i]:
                i += 1
                k += 1
                
            ret = (k+1)**2 + helper(boxes, i+1, j, 0, dp)
            for m in range(i+1, j+1):
                if boxes[m] == boxes[i]:
                    ret = max(ret, helper(boxes, i + 1, m - 1, 0, dp) + helper(boxes, m, j, k + 1, dp))
            
            dp[(i, j, k)] = ret
            return ret
            
        return helper(boxes, 0, len(boxes)-1, 0, {})
        

## 例子：Largest Sum Less Than K
一个数组list = [a1, a2, a3, a4];
- 对于largest sum的问题，可以很容易的有O(n)的解：使用类似dp的算法，以i结尾的largest sum为dp[i], dp[i] = max(dp[i-1] + list[i], list[i])
- 对于largest sum < k的问题，类似的s[i] 表示 sum(list[:i+1])，那么largetsum[i] = sum[i] - lowerbound(sum[i] - k); lowerbound在c++等里面可以用红黑树实现的数据结构来求解，对于python可以用bisect.bisect_left

In [0]:
import bisect
def LargestSumLessThanK(l, k):
  slist = [0]
  cursum = 0
  best = 0
  maxsum = -99999
  for n in l:
    cursum += n
    i = bisect.bisect_left(slist, cursum - k)
    if i < len(slist):
      best = cursum - slist[i]
    bisect.insort(slist, n)
    maxsum = max(maxsum, best)
  return maxsum
    
print(LargestSumLessThanK([1,2,-1,3,-4], 2))
print(LargestSumLessThanK([1,2,-1,3,-4,9, -5], 15))

## 例子：Wiggle Subsequence
https://leetcode.com/problems/wiggle-subsequence/description/

dp算法最重要的就是找到一个递推关系，很多情况下，递推关系并不能直接从题目中得到，需要稍微做一些变换。

这道题目的变换很有意思，因为是Wiggle，同时用了两个dp： up，down 分别表示看到i 为止（sequence 可以不包括i），最后一位是上升的sequence 和 最后是下降的 sequence 的长度

In [0]:
class Solution:
    def wiggleMaxLength(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        # up[] and down[] : max wiggle sequence length so far at index i.
        # 并且wiggle sequence里面最后一个分别是（和sequence前面相比）up和down的
        if not nums:
            return 0
        down, up = [0] *len(nums), [0] *len(nums)
        down[0] = 1
        up[0] = 1
        for i in range(1, len(nums)):
            if nums[i] > nums[i-1]:
                up[i] = down[i-1] + 1
                down[i] = down[i-1]
            elif nums[i] < nums[i-1]:
                down[i] = up[i-1] + 1
                up[i] = up[i-1]
            else:
                down[i] = down[i-1]
                up[i] = up[i-1]
        return max(down[-1], up[-1])

## 例子：Cherry Pickup

https://leetcode.com/problems/cherry-pickup/description/
这个例子很有意思，不能直接dp两遍，因为第一次的结果会影响第二次

这里有两个**重要**的思路：
1. 走几次是固定的，每条都是最短路，可以抽象出一个时间t的概念
2. 看成两个人一起走第一个人走到(r1, c1) 第二个人走到(r2, c2) 那么r1+c1 = r2 + c2 = t， 那么这里只要3个变量就够了

In [0]:
class Solution:
    def cherryPickup(self, grid):
        """
        :type grid: List[List[int]]
        :rtype: int
        """
        #这里有两个**重要**的思路：
        #1. 走几次是固定的，每条都是最短路，可以抽象出一个时间t的概念
        #2. 看成两个人一起走第一个人走到(r1, c1) 第二个人走到(r2, c2) 那么r1+c1 = r2 + c2 = t， 那么这里只要3个变量就够了
        #假设每次
        n = len(grid)
        #dp[i][j]表示在时间t的时候 一个人走到r1另一个人走到r2, 那么在t=2n的时候dp[n-1][n-1]就是要求的结果
        dp=[[[-9999]*(2*n-1) for _ in range(n)] for _ in range(n)] 
        dp[0][0][0] = grid[0][0]
        for t in range(1, 2*n-1):
            # 对于t时间，i至多在t, 至少在 t-n
            for i in range(max(0, t-n+1), min(n, t+1)):
                for j in range(max(0, t-n+1), min(n, t+1)):
                    # 当前分别在位置 (i, t-i) (j, t-j)
                    if grid[i][t-i] == -1 or grid[j][t-j] == -1:
                        continue
                        
                    # 上个时间i,j的位置，i==j那么grid[i][j]肯定已经被采过了
                    dp[i][j][t] = max(dp[a][b][t-1] for a, b in [(i-1,j-1), (i-1, j), (i, j-1), (i, j)] if 0<=a and 0<=b)
                    dp[i][j][t] += grid[i][t-i] + (grid[j][t-j] if i!= j else 0)
                    
        return  max(0, dp[-1][-1][-1])
                    
        

## 例子： New 21 Game
https://leetcode.com/problems/new-21-game/description/

In [0]:
class Solution:
    def new21Game(self, N, K, W):
        """
        :type N: int
        :type K: int
        :type W: int
        :rtype: float
        """
        # dp[i] 表示任何一次投掷获取到i的概率, 要求的就是sum(dp[K:N+1])
        # dp[0] = 1, dp[1] = 1/W ....
        # dp[i] = sum(last W dp values) / W
        # dp[i] = (d[i-1] + d[i-2] ...d[i-N])/W
        # 即 d[i] = sum(d[a]/W for a in range(j-W, j) if a>=0 and a<K)
        # 但是这里每次要求sum，效率比较低，那么可以slide window来简化
        if K == 0 or N >= K + W: return 1
        dp = [0]*(N+1)
        dp[0] = 1
        slidesum = 1
        for i in range(1, N+1):
            dp[i] = slidesum / W
            if i < K: slidesum += dp[i]
            if i - W >= 0: slidesum -= dp[i - W]
        return sum(dp[K:N+1])


## 例子：Race Car
https://leetcode.com/problems/race-car/description/
看分析 https://leetcode.com/problems/race-car/discuss/124326/Summary-of-the-BFS-and-DP-solutions-with-intuitive-explanation

# 回溯


## 例子：Sudoku Solver
https://leetcode.com/problems/sudoku-solver/discuss/


## 例子：N-Queens
https://leetcode.com/problems/n-queens/description/

## 例子：Wildcard Matching
https://leetcode.com/problems/wildcard-matching/description/

这也是一种回溯法，回溯的目标是*match的个数

In [0]:
class Solution:
    def isMatch(self, s, p):
        """
        :type s: str
        :type p: str
        :rtype: bool
        """
        si, pi = 0, 0 #表示当前遍历中 s的位置，p的位置
        matchsi, starpi = 0, -1 # 表示遍历过程中记录的状态，当前* match的 s 和p的位置，回溯使用
        while si < len(s):
          if pi < len(p) and (p[pi] == "?" or p[pi] == s[si]):
            si += 1
            pi += 1
          elif pi < len(p) and p[pi] == "*":
            starpi = pi
            matchsi = si
            pi += 1
          elif starpi != -1:
            pi = starpi + 1
            matchsi += 1
            si = matchsi
          else:
            return False
        while pi < len(p) and p[pi] == "*":
          pi +=1
        return pi == len(p)

## 例子：Expression Add Operators
问题的关键在于一种回溯技巧：保存当前加或者减的数，未来遇到乘，就把这个保存的数减或者加上，就是回溯掉之前的加减操作。

In [0]:
class Solution:
    def addOperators(self, num, target):
        """
        :type num: str
        :type target: int
        :rtype: List[str]
        """
        # 利用一种回溯法来加符号 2324 2+3 记住加了3 后面如果是乘号 2+3*2 那么 5-3+3*2 再次记住3*2=6
        def helper(ret, path, num, target, pos, val, multed):
            if pos == len(num):
                if val == target:
                    ret.append(path) #end here
                return
            else:
                for i in range(pos, len(num)):
                    if i != pos and num[pos] == "0":
                        return
                    curs = num[pos:i+1]
                    cur = int(curs)
                    if pos == 0:
                        helper(ret, path+curs, num, target, i+1, cur, cur)
                    else:
                        helper(ret, path+"+"+curs, num, target, i+1, val+cur, cur)
                        helper(ret, path+"-"+curs, num, target, i+1, val-cur, -cur)
                        helper(ret, path+"*"+curs, num, target, i+1, val-multed+multed*cur, multed*cur)
        
        if len(num) < 1:
            return []
        ret = []
        helper(ret, "", num, target, 0, 0, 0)
        return ret
        

# 遍历剪枝技巧


## Largest Rectangle in Histogram
https://leetcode.com/problems/largest-rectangle-in-histogram/discuss/

这个问题的stack应用方式很常见，遇到大的则push，遇到小的则pop直到满足条件，这种方式在 **转逆波兰式**这个例子里面也看到了，至于这种算法为什么正确，可以画图看一下，其实可以看成是一种剪枝策略

## Unique Substrings in Wraparound String

给一个字符串，找其中连续的字串个数，比如ab, abc, zab 都是连续字串。

Input: "zab"
Output: 6
Explanation: There are six substrings "z", "a", "b", "za", "ab", "zab" of string "zab" in the string s.

https://leetcode.com/problems/unique-substrings-in-wraparound-string/description/

这道题可以看成dp问题，但是最重要的是统计endwith的技巧

1. 首先切分成自问题，可以是startwith或者endwith某个字母的的substring个数为这个问题的子问题，那么只要统计startwith或者endwith某个字母的最长连续字串的长度
2. 统计endwith 某个字母 的 substring的最大长度比如 abcd endwith 'd' 的最大长度是4 那么endwith 'd' 的字串个数为4 （abcd, bcd, cd, d）
**endwith的思路(很常用)，如果是startwith，统计len会变得很麻烦
而统计endwith可以一边遍历，一边统计**

## Container With Most Water
https://leetcode.com/problems/container-with-most-water/solution/

这是一个双指针问题，遍历需要O(n^2), 使用剪枝可以变成O(n) 
剪枝就是要看没有必要遍历的部分，比如这里，我们双指针，长度短的那个以它为一条边的部门经过第一次之后就没有必要再遍历了，因为只会面积更小，不会更大了，所以短边可以往前走。

类似的问题还有：https://leetcode.com/problems/trapping-rain-water/description/

In [0]:
class Solution:
    def maxArea(self, height):
        """
        :type height: List[int]
        :rtype: int
        """
        left, right = 0, len(height) - 1
        area = 0
        while left < right:
          area = max(area, (right - left) * min(height[left], height[right]))
          if height[left] < height[right]:
            left+=1
          else:
            right-=1
        return area
          

In [0]:
print(Solution().maxArea([1,8,6,2,5,4,8,3,7]))

## Trapping Rain Water II

https://leetcode.com/problems/trapping-rain-water-ii/description/

这道题把[Trapping Rain Water](https://leetcode.com/problems/trapping-rain-water/description/)拓展到了三维空间，二维空间到Trapping Rain Water用双指针可以解决，但是要理解解决的本质是循环找小的，从小的往前推进，用这种思路拓展到三维空间，那么就可以用一个优先队列（堆）来解决这个问题。
解法：
https://www.youtube.com/watch?v=cJayBq38VYw


In [0]:
import heapq
class Solution:
    def trapRainWater(self, heightMap):
        """
        :type heightMap: List[List[int]]
        :rtype: int
        """
        # https://leetcode.com/problems/trapping-rain-water/description/
        # https://www.youtube.com/watch?v=cJayBq38VYw  
        if not heightMap or not heightMap[0]:
            return 0
        
        q = []
        m, n = len(heightMap), len(heightMap[0])
        visited = [[False]*n for _ in range(m)]
        for i in range(m):
            for j in range(n):
                if i == 0 or j == 0 or i == m-1 or j == n-1:
                    heapq.heappush(q, (heightMap[i][j], i, j))
                    visited[i][j]=True
        suma = 0
        while q:
            h, i, j = heapq.heappop(q)
            for x,y in [(i-1, j),(i+1, j),(i, j-1),(i, j+1)]:
                if 0<=x<m and 0<=y<n and not visited[x][y]:
                    suma += max(0, h - heightMap[x][y])
                    heapq.heappush(q, (max(heightMap[x][y],h), x, y))
                    visited[x][y] = True
                    
        return suma

## Maximum Subarray
https://leetcode.com/problems/maximum-subarray/description/

## Maximum Product Subarray
https://leetcode.com/problems/maximum-product-subarray/description/

## 三色旗问题

三路排序，可以解决三种数字排序，也可以在快速排序中使用。
这里举例两种算法，可以记住一种。

In [0]:
class Solution:
    # 这种算法比较好记: 
    #   for i in range(n): while swap(i, right--); while swap(i, left++)
    def sortColors(self, nums):
        """
        :type nums: List[int]
        :rtype: void Do not return anything, modify nums in-place instead.
        """
        left, right = 0, len(nums)-1 #分别是0 和2 应该放的位置
        for i in range(len(nums)):
            while nums[i] == 2 and i < right:
                nums[i], nums[right] = nums[right], nums[i]
                right-=1
            while nums[i] == 0 and i > left:
                nums[i], nums[left] = nums[left], nums[i]
                left+=1
        return nums
    
    def sortColors2(self, nums):
        left, right = 0, len(nums)-1 #分别是0 和2 应该放的位置
        i = left
        while i <= right:
            if nums[i] == 2:
                nums[i], nums[right] = nums[right], nums[i]
                right -=1
            elif nums[i] == 0:
                nums[i], nums[left] = nums[left], nums[i]
                left+=1
                i += 1
            else:
                i+=1
        return nums

In [0]:
print(Solution().sortColors([2,2,0,1,0]))
print(Solution().sortColors2([2,2,0,1,0]))

## Minimum Window Substring
https://leetcode.com/problems/minimum-window-substring/description/

这个解法里面比较重要的是不是一步到位的：

满足条件再 =》 收缩条件 =》 直到不满足条件 =》循环 这是遍历的一个常用技巧

针对这个题目，下面的算法对满足条件这件事用一个计数器进行计数，对单个字母也有一个map存计数器，那么重要的条件有：
1. if  tmap[a] > 0 => needcount --
2. if needcount==0 则满足条件，但是不是最优条件，begin可以再收缩: 收缩到一个位置 tmap[s[begin]] == 0 这下面一步，条件已经不满足了，needcount ++

In [0]:
from collections import defaultdict
class Solution:
    def minWindow(self, s, t):
        """
        :type s: str
        :type t: str
        :rtype: str
        """
        tmap = defaultdict(int)  # 需要多少个才能满足条件
        needcount = len(t)
        begin, end, minbegin, minwindow = 0, 0, 0, 9999999
        for i in t:
            tmap[i] += 1
        while end < len(s):
            if tmap[s[end]] > 0 :
                needcount -=1
            tmap[s[end]] -= 1
            end += 1
            while needcount == 0:
                if end - begin < minwindow:
                    minwindow = end - begin
                    minbegin = begin
                if tmap[s[begin]] == 0:
                    needcount += 1
                tmap[s[begin]] += 1
                begin += 1
        if minwindow == 9999999:
            return ""
        result = s[minbegin: minbegin + minwindow]
        return result
                    

## Candy
https://leetcode.com/problems/candy/description/

这是一个有趣的贪心问题，遍历两遍，第二遍遍历不会破坏第一遍遍历的结果，同时给大家的candy是贪心法最少的

## House Robber
https://leetcode.com/problems/house-robber/description/

**这是一个比较经典的问题**

可以用类似dp的思路来解, dp[i] 表示偷到第i家，
dp[i] = max(dp[i-2] + n[i], dp[i-1])
对这个公式可能的疑问是dp[i-1]的时候可能也会偷第i家的，因为对于dp[i-1]可能i-1也没偷，其实这种情况dp[i-1] 就等于 dp[i-2]了，所以这个公式是没问题的。


## Longest Consecutive Sequence
https://leetcode.com/problems/longest-consecutive-sequence/description/

对于连续数字abc. 出现次序为 a...b...c 或者 a....c....b, 采用一种左右扩展的策略确定能都合并所有序列

```
[left1...right1] n [left2...right2]
 
  right1-left1         right2-left2
  
 => 
 
 [left1...right1] n [left2...right2]
 
  right2-left2 + 1 + right2-left2

```


In [0]:
import collections
class Solution:
    def longestConsecutive(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        recodmap = {}
        maxn = 0
        for n in nums:
            if n not in recodmap:
                left = recodmap[n-1] if n-1 in recodmap else 0   #以left为一个端点的length
                right = recodmap[n+1]  if n+1 in recodmap else 0 #以right为一个端点的length
                lengthn = left + right + 1
                maxn = max(lengthn, maxn)
                recodmap[n] = lengthn
                recodmap[n - left] = lengthn  # 更新另一个端点
                recodmap[n + right] = lengthn # 更新另一个端点
        return maxn
        

## Search a 2D Matrix II
https://leetcode.com/problems/search-a-2d-matrix-ii/description/

从哪里开始遍历对这题很重要

##  Longest Substring with At Least K Repeating Characters
https://leetcode.com/problems/longest-substring-with-at-least-k-repeating-characters/description/

1. divider and conquer的方法是比较容易想出来的，用不足k的的字母作为分割点，来divide

In [0]:
import collections
class Solution:
    def longestSubstring1(self, s, k):
        """
        :type s: str
        :type k: int
        :rtype: int
        """
        for c in set(s):
            if s.count(c) < k:
                return max(self.longestSubstring(s1, k) for s1 in s.split(c))
        return len(s)
    
    def longestSubstring(self, s, k):
        """
        :type s: str
        :type k: int
        :rtype: int
        """
        # 算法的原理是 假设unique=2, k=5, 对于一个s满足条件，那么就是2个字母，每个重复大余5次
        # 用window来统计noLessThanK的个数，unique < h -> j++; unique > h -> i++
        # 如果noLessThanK=unique那么满足条件，
        maxl = 0
        for h in range(1, 27):
            counts = collections.defaultdict(int)
            i, j = 0, 0
            unique, noLessThanK = 0, 0
            while j < len(s) :
                if unique <= h:
                    if counts[s[j]] == 0:
                        unique += 1
                    counts[s[j]] += 1
                    if counts[s[j]] == k:
                        noLessThanK += 1
                    j +=1
                else:
                    if counts[s[i]] == k:
                        noLessThanK -= 1
                    counts[s[i]] -= 1
                    if counts[s[i]] == 0:
                        unique -= 1
                    i += 1
                if unique == h and unique == noLessThanK:
                    maxl = max(j - i, maxl)
  
        return maxl

## KMP算法
https://www.zhihu.com/question/21923021

In [0]:
# ！！！！nexttable是最大相同前缀后缀长度！！！！
# 记忆方式 if j == -1 or s0[i] == s1[j]: 
#            j++,i++, (next[i]=j 求next table 有这句，否则没有）
#         else j=next[j]
def nextTable(s0):
  i, j = 0, -1
  next = [0] * (len(s0) + 1)
  next[0] = -1         

  while i < len(s0):
    if j == -1 or s0[i] == s0[j]:
      i += 1  
      j += 1
      next[i] = j
    else:
      j = next[j]
  
  return next

def KMP(s0, s1):
  i, j = 0, -1
  next = nextTable(s0)       

  while i < len(s0) and j < len(s1):
    if j == -1 or s0[i] == s1[j]:
      i += 1  
      j += 1
    else:
      j = next[j]
  
  return i - j if j == len(s1) else -1


In [0]:
print(nextTable("abababca"))
print(nextTable("aababca"))
print(KMP("abc", "bc"))
print(KMP("abc", "bcd"))
print(KMP("acdcdef", "cde"))

### shortest-palindrome
kmp的一个应用，这个主要是用了nextTable，需要理解**nextTable里面记录的是最大相同前缀后缀长度** abcdxxxxxxabcd = > 4
https://leetcode.com/problems/shortest-palindrome/description/

**回文问题的常见解决办法：nexttable, dp, manacher; #todo:在leetcode找题目总结一下**


In [0]:
class Solution(object):
    def shortestPalindrome(self, s):
        """
        :type s: str
        :rtype: str
        """
        # abcb#bcba => abcba 
        # dcba#abcd  => dcbbcd
        # catacb#bcatac => bcatac[:l - 5] ) + catacb  = bcatacb
        def nextTable(s0):
          i, j = 0, -1
          next = [0] * (len(s0) + 1)
          next[0] = -1         

          while i < len(s0):
            if j == -1 or s0[i] == s0[j]:
              i += 1  
              j += 1
              next[i] = j
            else:
              j = next[j]
          return next
        
        s0 = s + "#" + s[::-1]
        nexttable = nextTable(s0)
        return s[::-1][:len(s) - nexttable[-1]] + s

### Repeated Substring Pattern
https://leetcode.com/problems/repeated-substring-pattern/description/

In [0]:
class Solution(object):
    def repeatedSubstringPattern1(self, s):
        """
        :type s: str
        :rtype: bool
        """
        # 方法1:可以证明
        # S = a1,a2...an
        # SS = a1,a2..an,a1,a2..an
        # SS[1:-1] = a2...an,a1,a2...an-1
        # SS[1:-1] contains S =>  S = ak, ak+1...an,..ak-1 == a1,a2...an 而且 k!=1
        # 有repeat pattern
        return s in (s+s)[1:-1]
    
    def repeatedSubstringPattern2(self, s):
        # 方法2:思路类似找最大前缀后缀长度
        # 比如 abcabcabc 对应next表为 0 000123456 
        # 那么 有repeat pattern <=> next[-1] != 0 而且 n-next[-1]为pattern len%pattern == 0
        i, j = 0, -1
        n = len(s)
        next = [0]*(n + 1)
        next[0] = -1 
        while i < n:
            if j == -1 or s[i] == s[j]:
                i, j = i+1, j+1
                next[i] = j
            else:
                j = next[j]
        return next[-1] !=0 and next[-1]%(n-next[-1])==0
        
    def repeatedSubstringPattern3(self, s):   
        # 方法3:使用正则表达式
        import re
        return re.match(r"^([a-z]+)\1+$", s) != None

### 判断旋转词
如果对于一个字符串A，将A的前面任意一部分挪到后边去形成的字符串称为A的旋转词。

比如A=”12345”,A的旋转词有”12345”,”23451”,”34512”,”45123”和”51234”。

对于两个字符串A和B，请判断A和B是否互为旋转词。

这个题可以用KMP解，AB为旋转词 <=> 'AA'.contains('B')

# Greedy
贪心算法通常需要证明

## Best Time to Buy and Sell Stock II
https://leetcode.com/problems/best-time-to-buy-and-sell-stock-ii/description/

这道题可以理解成一种贪心算法解决：遇到p>prev 就 profit += p - prev

为什么这种算法是对的，只关注递增的数字对：
- 假设 a..b..c, a < b < c, 那么 c-b + b -a = c-a 为最优解
- 假设 a..b, c..d, a<b, c<d, b>c, 那么b-a+d-c > d-a 为最优解

## Best Time to Buy and Sell Stock IV
用dp来解决，dp(i,j) is the max profit for up to i transactions by time j (0<=i<=K, 0<=j<=T).

```python
# dp[i, j] represents the max profit up until prices[j] using at most i transactions. 
# dp[i, j]=max(dp[i, j-1], prices[j] - prices[jj] + dp[i-1, jj]) { jj in range of [0, j-1] }
#         =max(dp[i, j-1], prices[j] + max(dp[i-1, jj] - prices[jj]))
 

# hold 是时间j时，i-1次transactions后再一次买入操作之后的max
# dp[i][j] 则是时间j时i次transactions的max
# 相当于把dp分成了两块
hold =  -prices[0];
for j in range(1, T)
  dp[i][j] = max(dp[i][j - 1], prices[j] + hold);
  hold = max(hold, dp[i - 1][j - 1] - prices[j]);
```

## 例子（重要）：Longest Increasing Subsequence
https://leetcode.com/problems/longest-increasing-subsequence/description/

dp的解法为O(n^2), 不展开
另外一种解法可以做到O(nlgn), 做法是这样：维护一个队列，这个队列表示长度为i+1的队列的tail:
```
2 1 5 3 6 4 8 9 7
---------------------
2 : tail = [2] ：长度为2的增序队列的tail为2
1 : tail = [1] ：长度为2的增序队列的tail为1, 这里就是贪心策略了，替换了2为1，长度还是1，但是tail为1是为了之后的增序
5 : tail = [1, 5]
3 : tail = [1, 3]
6 : tail = [1, 3, 6]
4 : tail = [1, 3, 4]
8 : tail = [1, 3, 4, 8]
9 : tail = [1, 3, 4, 8, 9]
7 : tail = [1, 3, 4, 7, 9]

!!!!! 注意。这个1,3,4,7,9不是LIS，它只是存储的对应长度LIS的最小末尾。而且这个长度是和LIS长度相同

这个替换Tail的操作可以用二分查找实现，所以是O(nlgn)
```


另一条类似的题目：
https://leetcode.com/problems/increasing-triplet-subsequence/description/

这道题目的思路也很重要，算是 Longest Increasing Subsequence的应用程序，本质的是**Find Longest Increasing Pair**  那么可以先sort，然后再Find Longest Increasing Subsequence，排序的时候可以使用一点小技巧，把pair[0]相等，但是pair[1]大的放在前面，这样的好处是在后面一步里面就自动把这个排除了：
https://leetcode.com/problems/russian-doll-envelopes/description/

In [0]:
class Solution:
    def lengthOfLIS1(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        if not nums:
            return 0
        dp = [1] * len(nums)
        maxn = 0
        for i in range(len(nums)):
            dp[i] = max([1] + [dp[k] + 1 for k in range(0, i) if nums[i] > nums[k]])
            maxn = max(maxn, dp[i])
        return maxn
    
    def lengthOfLIS(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        if not nums:
            return 0
        tail = [nums[0]]
        for n in nums[1:]:
            # find lowerbound
            l, r = -1, len(tail)
            while l + 1 < r:
                mid = (l + r) // 2
                if tail[mid] < n:
                    l = mid
                else:
                    r = mid
            if l + 1 >= len(tail):
                tail.append(n)
            else:
                tail[l + 1] = n
        return len(tail)

## 例子： Patching Array
https://leetcode.com/problems/patching-array/description/
贪心算法
```
1, 2, 5, 11
1: 覆盖到 1
2: 覆盖到 3
5: 前面覆盖到3 加4 那么可以覆盖到7， 加5 覆盖到11了


换一个角度
1: 覆盖到 1 缺 2
2: 2<=2, 覆盖到3，缺4
5: 5>4, 补4，覆盖到7+5，12，缺13
```

In [0]:
class Solution:
    def minPatches(self, nums, n):
        """
        :type nums: List[int]
        :type n: int
        :rtype: int
        """
        miss = 1
        added, i = 0, 0
        while miss <= n:
            if i < len(nums) and nums[i] <= miss:
                miss += nums[i]
                i += 1
            else:
                miss += miss
                added += 1
        return added

## 例子：Course Schedule III
https://leetcode.com/problems/course-schedule-iii/description/

In [0]:
import heapq
class Solution:
    def scheduleCourse(self, courses):
        """
        :type courses: List[List[int]]
        :rtype: int
        """
        courses = sorted(courses, key=lambda a: a[1])
        queue, start = [], 0
        for d, e in courses:
            start += d
            heapq.heappush(queue, -d)
            if start > e:
            # 如果有能替换它的 让结束时间能满足的，就替换，没有的话，这个又会重新被pop出来
            # 所以这里用if就够了 不用while
                start += heapq.heappop(queue)
        return len(queue)

# 递归/Divide And Conquer
递归也可以看成divide and conquer，把大问题变成更小规模的子问题

 ## 例子：Different Ways to Add Parentheses

In [0]:
class Solution:
    def diffWaysToCompute(self, input):
        """
        :type input: str
        :rtype: List[int]
        """
        ret = []
        for i, a in enumerate(input):
            if a in {"+", "-", "*"}:
                leftlist, rightlist = self.diffWaysToCompute(input[:i]), self.diffWaysToCompute(input[i+1:])
                for left in leftlist:
                    for right in rightlist:
                        if a == "+":
                            ret.append(left + right)
                        elif a == "-":
                            ret.append(left - right)
                        else:
                            ret.append(left * right)
        if not ret:
            ret.append(int(input))
        return ret

## 例子：Remove Duplicate Letters
https://leetcode.com/problems/remove-duplicate-letters/description/

In [0]:
class Solution:
    def removeDuplicateLetters(self, s):
        count = [0]*26	
        for a in s:
            count[ord(a)-ord('a')]+=1#统计每个字符出现次数

        pos = 0
        for i in range(len(s)):
            if s[i] < s[pos]:
                pos = i
            count[ord(s[i])-ord('a')]-=1
            if count[ord(s[i])-ord('a')] == 0:
                break
        # if s:
        #   print(s, count, s[pos])
        return "" if len(s)==0 else s[pos] + self.removeDuplicateLetters(s[pos+1:].replace(s[pos],''))

In [0]:
print(Solution().removeDuplicateLetters("cbacdcbc"))
print(Solution().removeDuplicateLetters("cbacdcbcd"))

## 例子：Split Array Largest Sum
https://leetcode.com/problems/split-array-largest-sum/description/

二分查找一个max，这个max的作用是定义cut：
1. 如果sum<= max -> cut
2. 如果n[i] > max -> 不能cut
3. 如果刚好cut > 指定次数 -> False

In [0]:
class Solution:
    def splitArray(self, nums, m):
        """
        :type nums: List[int]
        :type m: int
        :rtype: int
        """
        def work(nums, maxn, cut):
            acc = 0
            for n in nums:
                if n > maxn:
                    return False
                elif acc + n <= maxn:
                    acc += n
                else:
                    cut -= 1
                    acc = n
                    if cut < 0:
                        return False
            return True
        
        left = max(nums)
        right = sum(nums)
        while left < right:
            mid = (left + right) //2
            if work(nums, mid, m-1):
                right = mid
            else:
                left = mid + 1
                
        return left
        
        

## 例子：Scramble String
https://leetcode.com/problems/scramble-string/description/

这道题能找到递归的思路就很容易下手了
```

isScramble(s1, s2)=isScramble(s1[:i], s2[:i]) and isScramble(s1[i:], s2[i:])\
                      or isScramble(s1[:i], s2[-i:]) and isScramble(s1[i:], s2[:-i]) \
                                    for i in range(len)
```

## 例子：Special Binary String

https://leetcode.com/problems/special-binary-string/description/

```python
class Solution:
    def makeLargestSpecial(self, S):
        """
        :type S: str
        :rtype: str
        """
        # Special binary strings
        # 第一个字符肯定是1 最后一个字符肯定是0 
        # 如何prefix是 Special binary strings 那么后缀肯定也是Special binary strings (连续，可以直接sort)
        # 一个Special binary strings a 要么可以拆成几个连续的Special binary strings 要么 a[1:-1]是Special binary strings
        # 所以不管怎么样都是可以变成更小的子问题的
        # 这个问题也可以看成是括号的问题 (()) 或者()()
        c1, i = 0, 0
        l = []
        for j, a in enumerate(S):
            c1 += 1 if a == "1" else -1
            if c1 == 0:
                l.append("1" + self.makeLargestSpecial(S[i+1:j]) + "0")
                i = j+1
        return "".join(sorted(l, reverse=True))
```

## 例子：K-th Smallest Prime Fraction
https://leetcode.com/problems/k-th-smallest-prime-fraction/description/
**这是个很好的例子，当二分的条件不再明显的时候，如果构造出这样的边界和条件**

看这个总结

https://leetcode.com/problems/k-th-smallest-prime-fraction/discuss/115819/Summary-of-solutions-for-problems-%22reducible%22-to-LeetCode-378

https://leetcode.com/problems/find-k-th-smallest-pair-distance/discuss/109082/Approach-the-problem-using-the-%22trial-and-error%22-algorithm

In [0]:
import bisect
class Solution:
    def kthSmallestPrimeFraction(self, A, K):
        """
        :type A: List[int]
        :type K: int
        :rtype: List[int]
        """
        l, r, N = 0, 1, len(A)
        while True:
            mid = (l + r)/2
            # 固定a 找b 然 a/b<=mid 即 b>=a/mid
            border = [bisect.bisect_left(A, a/mid) for a in A]
            count_lesseq_than_mid = sum(N-i for i in border)
            if count_lesseq_than_mid < K:
                l = mid
            elif count_lesseq_than_mid > K:
                r = mid
            else:
                # 寻找距离border 最近的里面最大的
                return max([[A[i], A[j]] for (i, j) in enumerate(border) if j < N],  key=lambda x: x[0] / x[1])
            
            

## 例子：Rotated Digits
https://leetcode.com/problems/rotated-digits/description/

In [0]:
import bisect
class Solution:
    # 这种算法值得记住，很好理解 也比较常见，这里面还可以加memo table用于加速，这里省略了
    def rotatedDigits(self, N):
        """
        :type N: int
        :rtype: int
        """
        a = [0, 1, 2, 5, 6, 8, 9]
        b = [0, 1, 8]
        
        # 数字取candis里面的值，组成的 1-N 范围的数字 数量
        def helper(candis, N):
            
            NS = str(N)
            if len(NS) == 1:
                i = bisect.bisect_left(candis, N)
                if i < len(candis) and candis[i] == N:
                    i += 1
                return i
            first = int(NS[0])
            i = bisect.bisect_left(candis, first)
            # 如果第一位取的数字小于first，那么后面都可以取 9 
            # 如果第一位取的数字等于first，那么后面的算int(NS[1:]) 就可以
            b = (10**(len(NS) - 1)) - 1
            if i < len(candis) and candis[i] == first:
                a =  i*helper(candis, b) + helper(candis, int(NS[1:]))
                return a 
            else:
                a =  i*helper(candis, b)
                return a
                
            
        return helper(a, N) - helper(b, N)
        
        

# 经典算法
值得记住


## 卡特兰数问题的变种
https://zh.wikipedia.org/wiki/%E5%8D%A1%E5%A1%94%E5%85%B0%E6%95%B0

### Generate Parentheses
https://leetcode.com/problems/generate-parentheses/description/

卡特兰数问题的解个数为C(2n, n)/(n+1)
比如 n =3, 解为6!/(3!*3!*4) = 5


卡特兰数问题一般都可以用**递归**或者**dp**(如果只要求个数，而不是具体的解)来解，再比如这道题
[Unique Binary Search Trees](https://leetcode.com/problems/unique-binary-search-trees-ii/description/)

In [0]:
class Solution:
    def generateParenthesis(self, n):
        """
        :type n: int
        :rtype: List[str]
        """
        def helper(left, right, s, slist):
          if left == 0 and right == 0:
            slist.append(s)
          if left > 0:
            helper(left-1, right, s + "(", slist)
          if right > 0 and left < right:
            helper(left, right - 1, s + ")", slist)
        
        alist = []
        helper(n, n, "", alist)
        return alist

print(Solution().generateParenthesis(3))

## FindTarget/LowerBound/UpperBound
这几个算法要熟记，常是其他算法的子程序
实现方式比较多，建议熟记一种，直接写出来：

|算法|初始化| 更新方式 | 返回 |
|---|--|--|--|
|FindTarget|start, end = 0, len -1| start = mid + 1/ end = mid - 1/ return mid| return -1
|LowerBound|start, end = 0, len|start = mid+1/ end = mid |return start
|UpperBound|start, end = 0, len|start = mid+1/ end = mid | return start


In [0]:
# 这几个个函数可以当成模版记住！！
# 这是传统的实现方式 『当target只有一个的时候』：
def findTarget(list, target):
    start = 0
    end = len(list) - 1
    # 注意这里是end = len(a)-1; <=
    while start <= end:
        mid = (start + end)//2
        if list[mid] < target:
            start = mid + 1
        elif list[mid] > target:
            end = mid - 1
        else:
            return mid
    return -1

# >= target的最小值
def lowerBound(a, x):
#def bisect.bisect_left(a, x):
    lo, hi = 0, len(a)
    # 注意这里hi = len(a); <
    while lo < hi:
        mid = (lo+hi)//2
        if a[mid] < x: lo = mid+1
        else: hi = mid
#     return lo #这里从bisect改成这样是为了防御极端情况
    return lo if lo<len(a) and a[lo] >= x else -1



# > target的最小值
def upperBound(a, x):
#def bisect.bisect_right(a, x):
    lo, hi = 0, len(a)
    while lo < hi:
        mid = (lo+hi)//2
        if x < a[mid]: hi = mid
        else: lo = mid+1
#     return lo #这里从bisect改成这样是为了防御极端情况
    return lo if lo<len(a) and a[lo] > x else -1


  

### Find First and Last Position of Element in Sorted Array
https://leetcode.com/problems/find-first-and-last-position-of-element-in-sorted-array/description/

本质是find upperbound 和 find lowerbound的一点点变化

In [0]:

# >= target的最小值
def lowerBound(a, x):
#def bisect.bisect_left(a, x):
    lo, hi = 0, len(a)
    while lo < hi:
        mid = (lo+hi)//2
        if a[mid] < x: lo = mid+1
        else: hi = mid
#     return lo #这里从bisect改成这样是为了防御极端情况
    return lo if lo<len(a) and a[lo] >= x else -1



# > target的最小值
def upperBound(a, x):
#def bisect.bisect_right(a, x):
    lo, hi = 0, len(a)
    while lo < hi:
        mid = (lo+hi)//2
        if x < a[mid]: hi = mid
        else: lo = mid+1
#     return lo #这里从bisect改成这样是为了防御极端情况
    return lo if lo<len(a) and a[lo] > x else -1


# = target的最小值
def lowerBoundEqual(a, x):
    lo, hi = 0, len(a)
    while lo < hi:
        mid = (lo+hi)//2
        if a[mid] < x: lo = mid+1
        else: hi = mid
    return lo if lo<len(a) and a[lo] == x else -1

# = target的最大值
def upperBoundEqual(a, x):
    lo, hi = 0, len(a)
    while lo < hi:
        mid = (lo+hi)//2
        if x < a[mid]: hi = mid
        else: lo = mid+1
    return lo-1 if 0<=lo-1<len(a) and a[lo-1] == x else -1


class Solution:
    def searchRange(self, nums, target):
        """
        :type nums: List[int]
        :type target: int
        :rtype: List[int]
        """
        l = lowerBoundEqual(nums, target)
        r = upperBoundEqual(nums, target)
        return [l, r]

## MinMax Problem

https://en.wikipedia.org/wiki/Minimax

这个guess-number问题需要换个角度看成一个最优决策问题，那么算法就比较容易了：
https://leetcode.com/problems/guess-number-higher-or-lower-ii/description/

假设不管我猜什么数值，对手总是能改数字的，而他的目标是让我的损失最大，那么我的目标就是在我的决策中取 min(所有的max)

```
function minimax(node, depth, maximizingPlayer) is
    if depth = 0 or node is a terminal node then
        return the heuristic value of node
    if maximizingPlayer then
        value := −∞
        for each child of node do
            value := max(value, minimax(child, depth − 1, FALSE))
        return value
    else (* minimizing player *)
        value := +∞
        for each child of node do
            value := min(value, minimax(child, depth − 1, TRUE))
        return value
```

In [0]:
class Solution(object):
    def getMoneyAmount(self, n):
        """
        :type n: int
        :rtype: int
        """
        dp = [[0] *(n+1) for _ in range(n+1)]
        def minMoneyAmount(lower, upper):
            if lower >= upper:
                return 0
            if dp[lower][upper]:
                return dp[lower][upper]
            else:
                minm = min( max(minMoneyAmount(lower, i-1), minMoneyAmount(i+1, upper)) + i \
                                for i in range(lower, upper+1)  )
                dp[lower][upper] = minm
                return minm
            
        return minMoneyAmount(1, n)
        

## Sliding Window Problem


普遍可以用一种类似槽位放珠子的思路来解这种题目，重点看下面的第一个例子

https://leetcode.com/problems/longest-substring-without-repeating-characters/
https://leetcode.com/problems/substring-with-concatenation-of-all-words/
https://leetcode.com/problems/longest-substring-with-at-most-two-distinct-characters/
https://leetcode.com/problems/find-all-anagrams-in-a-string/

### [Find All Anagrams in a String](https://leetcode.com/problems/find-all-anagrams-in-a-string/description/)

找出s中包含p的anagram的index位置，比如 s: "cbaebabacd" p: "abc"
返回[0, 6]， 对应位置分别有"cba", "bac"

In [0]:
import collections
class Solution(object):
    def findAnagrams(self, s, p):
        """
        :type s: str
        :type p: str
        :rtype: List[int]
        """
        # 用珠子放进槽位的思路来想这道题：
        # 如 p=aba 的槽位为 [ABA], 另外还有一个无效槽位区域[ABCD...Z]用来放没用的珠子
        # 我们需要实现的是 所有的有效槽位[ABA]都被珠子放满了
        # 槽位可以用dict或者array来实现, diff表示目前还差几个槽位没放满的
        d, ret = collections.defaultdict(int), []
        diff = n = len(p)
        for a in p:
            # -1表示有一个槽位需要被填充
            d[a] -= 1
        for i in range(len(s)):
            # left, right表示左右指针, right为要放进来的，left为要拿出去的，right - left = n
            left, right = i - n, i
            # 拿出left, <=0 说明拿出的是有效区的珠子,diff+1
            if left >= 0:
                if d[s[left]] <= 0:
                    diff += 1
                d[s[left]] -= 1

            # 放入right, 如果放的是有效区，那么diff-1
            d[s[right]] += 1
            if d[s[right]] <=0:
                diff -= 1
            
            if diff == 0:
                # append left
                ret.append(left+1) 
                
        return ret

### [Longest Substring Without Repeating Characters](https://leetcode.com/problems/longest-substring-without-repeating-characters/description/)

如题，比如"abcabcbb"返回 3， "abc" 就是最长的没有repeating的

In [0]:
class Solution(object):
    def lengthOfLongestSubstring(self, s):
        """
        :type s: str
        :rtype: int
        """
        # 还是用槽里面放珠子的思路来想这道题
        d = set()
        left, right = 0, 0
        maxlen = 0
        while right < len(s):
            # 槽空的，可以放
            if s[right] not in d:
                maxlen = max(maxlen, right-left+1)
                d.add(s[right])
                right += 1
            else:
                # 不能放了，先腾空
                while s[right] in d:
                    d.remove(s[left])
                    left += 1
        
        return maxlen

### [Minimum Window Substring](https://leetcode.com/problems/minimum-window-substring/)

找S中包含T中所有字母的最短substring. 如 S = "ADOBECODEBANC", T = "ABC" -> "BANC"

In [0]:
class Solution(object):
    def minWindow(self, s, t):
        """
        :type s: str
        :type t: str
        :rtype: str
        """
        
        d, ret = collections.defaultdict(int), []
        diff = len(t)
        for a in t:
            # -1表示有一个槽位需要被填充
            d[a] -= 1
        left, right, minlen, minstr = 0, 0, float("inf"), ""
        while right < len(s):
            # left, right表示左右指针, right为要放进来的，left为要拿出去的
            # 放入right, 如果放的是有效区，那么diff-1
            
            if diff >0:
                d[s[right]] += 1
                if d[s[right]] <=0:
                    diff -= 1
                right += 1
                
            while diff == 0:
                # append left
                if right - left < minlen:
                    minlen = right - left
                    minstr = s[left: right]
                
                # 拿出left, <=0 说明拿出的是有效区的珠子,diff+1
                if d[s[left]] <= 0:
                    diff += 1
                d[s[left]] -= 1
                left += 1
                
        return minstr

### [Substring with Concatenation of All Words](https://leetcode.com/problems/substring-with-concatenation-of-all-words/description/)

words里面的单词长度相同，用words组合成的string是s的substring，求index
  s = "barfoothefoobarman",
  words = ["foo","bar"]
Output: [0,9]

## 蓄水池抽样
https://blog.csdn.net/Hackbuteer1/article/details/7971328
很大的数据流里面抽取k个，使得所有数据被抽到的概率相同

```python
# 初始化：reservoir = nums[:k]
for i = k+1 to ...
   M=random(1, i);
   if( M < k):
     SWAP(reservoir[k], nums[i])
```

一个应用的例子：https://leetcode.com/problems/linked-list-random-node/description/

## 布隆过滤器

```python
# init
for url in urls:
  for hash in hashes:
    hash_bit_table[hash[url]] = 1  
    
# query
return all([hash_bit_table[hash(url)] for hash in hashes])
```

## Boyer-Moore Majority Vote 
https://www.jianshu.com/p/dfd676b71ef0

## Manacher算法：求最大长度回文字串的线性算法


```
原始回文串是S，处理之后为S'
1. 维护right(当前最大扩展到的右边界), center(扩展到右边界的时候的中心位置)和Z(以i为中心的最大回文串S'半径)
2. 当i < MaxRight的时候：min(right - i, Z[2 * center - i])，然后扩展

```

应用：

https://leetcode.com/problems/palindromic-substrings/description/



In [0]:
def manacher(s):
    #预处理
    A = '@#' + '#'.join(s) + '#$'
    Z = [0] * len(A)
    center = right = 0
    for i in range(1, len(A) - 1):
        if i < right:
            Z[i] = min(right - i, Z[2 * center - i])
        # 左右扩展
        while A[i + Z[i] + 1] == A[i - Z[i] - 1]:
            Z[i] += 1
        # 更新 center, right
        if i + Z[i] > right:
            center, right = i, i + Z[i]
    return Z
    # 如果求最大长度
    #return max(Z)

In [4]:
manacher("abba")

[0, 0, 1, 0, 1, 4, 1, 0, 1, 0, 0]

## Next Permutation
https://leetcode.com/problems/next-permutation/
1. 找最大index k 满足 nums[k] < nums[k + 1]. 如果没有直接reverse
比如 of [3, 2, 1] -> [1, 2, 3].
2. 找最大 index l > k 满足 nums[k] < nums[l].
3. 交换 nums[k],  nums[l].
4. 逆序 nums[k + 1：] 

如 132  k=1 l=2  swap: 231 reverse: 213


In [0]:
class Solution(object):
    def nextPermutation(self, nums):
        """
        :type nums: List[int]
        :rtype: void Do not return anything, modify nums in-place instead.
        """

        k = -1; l = -1
        for i in range(len(nums)):
            if i + 1 < len(nums) and nums[i] < nums[i + 1] :
                k = i
            if k != -1 and nums[i] > nums[k] and i > l:
                l = i
        if k == -1:
            nums.reverse()
        else:
            nums[k], nums[l] = nums[l], nums[k]
            nums[k+1:] = reversed(nums[k+1 :])
        return nums

## 几何经典算法
找fence:
https://leetcode.com/problems/erect-the-fence/description/
https://leetcode.com/problems/erect-the-fence/solution/

判断是否在多边形中：
https://www.jianshu.com/p/3187832cb6cc

### 找fence: Monotone_Chain_Convex_Hull
http://www.algorithmist.com/index.php/Monotone_Chain_Convex_Hull

In [0]:
from functools import reduce
class Solution:
    def outerTrees(self, points):
        """
        :type points: List[Point]
        :rtype: List[Point]
        """
        def counter_clockwise(a1, a2, a3):
            return (a2[1] - a1[1])*(a3[0] - a2[0]) <= (a3[1] - a2[1])*(a2[0] - a1[0])
        
        def drive(hull, r):
            hull.append(r)
            while len(hull) > 2 and not counter_clockwise(*hull[-3:]):
                hull.pop(-2)
            return hull
        points.sort(key = lambda p: (p[0], p[1]))
        lower = reduce(drive, points, [])
        upper = reduce(drive, points[::-1], [])
        #print(lower, upper)
        return [[a, b] for a, b in list(set((a, b) for a, b in lower + upper))]
        

## 图着色问题
```
图着色问题（英语：Graph Coloring Problem，简称GCP），又称着色问题，是最著名的NP-完全问题之一[1]。
给定一个无向图 {\displaystyle G=(V,E)} G=(V,E)，其中 {\displaystyle V} V为顶点集合， {\displaystyle E} E为边集合，图着色问题即为将 {\displaystyle V} V分为K个颜色组，每个组形成一个独立集，即其中没有相邻的顶点。其优化版本是希望获得最小的K值[2]。
```

图的着色问题可以用回溯法来解

也经常会使用贪心算法，但是贪心算法不能保证得到最优解，只能保证得到还不错的解。

### 例子：Is Graph Bipartite
https://leetcode.com/problems/is-graph-bipartite/description/

下面给了两种解法，都需要记住

In [0]:
class Solution:
    # 这种解法是经典的解法，但是对于这道题会LTE
    def isBipartite1(self, graph):
        """
        :type graph: List[List[int]]
        :rtype: bool
        """
        # 这是一个图着色问题，想用两种颜色把图着色
        N = len(graph)
        color_table = [0] * N # 颜色从1开始，0表示还没有颜色
        
        # 给顶点v涂色c 合法吗
        def colorValid(color_table, c, v):
            for neighbor in graph[v]:
                if color_table[neighbor] == c:
                    return False
            return True
        
        
        # m是可以用的颜色数量，v是当前标记的顶点
        def colorUtil(color_table, m, v):
            if v == N:
                return True
            # 逐次尝试
            for c in range(1, m+1):
                if colorValid(color_table, c, v):
                    color_table[v] = c
                    if colorUtil(color_table, m, v+1):
                        return True
                    #回溯
                    color_table[v] = 0
                    
            return False
        
        return colorUtil(color_table, 2, 0)
    
    # 由于只有两种颜色，可以遍历的同时染色
    def isBipartite(self, graph):
        """
        :type graph: List[List[int]]
        :rtype: bool
        """
        # 这是一个图着色问题，想用两种颜色把图着色
        N = len(graph)
        color_table = [0] * N # 0表示还没有颜色, 颜色为1,-1
        
        
        # m是可以用的颜色数量，v是当前标记的顶点
        def dfs(color_table, v):
            # 逐次尝试
            for i in graph[v]:
                if not color_table[i]:
                    color_table[i] = -color_table[v]
                    if not dfs(color_table, i):
                        return False
                elif color_table[v] == color_table[i]:
                    return False
            return True
        
        for i in range(N):
            if not color_table[i]:
                color_table[i] = 1
                if not dfs(color_table, i):
                    return False
                    
        return True

## 随机洗牌算法
生成一个0 - n-1的序列，序列顺序随机

In [0]:
import random
def perm(n):
    nums = list(range(n))
    for i in range(n):
        randindex = random.randint(i, n-1)
        nums[i], nums[randindex] = nums[randindex], nums[i]
    return nums

### 例子: Random Flip Matrix

https://leetcode.com/problems/random-flip-matrix/description/
这是随机洗牌算法的一个巧妙改进：如何在节约空间和时间的前提下使用随机洗牌算法：本质还是随机洗牌，返回start数字，和start交换的那个数字可能不是n-1了 （已经被交换过了)，用d来记录变化后的数值


In [0]:
class Solution(object):
    def __init__(self, n_rows, n_cols):
        """
        :type n_rows: int
        :type n_cols: int
        """
        self.n_rows = n_rows
        self.n_cols = n_cols
        self.start = 0
        self.end = n_rows*n_cols - 1
        self.d = {}
        
    def flip(self):
        """
        :rtype: List[int]
        """
        rand = random.randint(self.start, self.end)
        ret = self.d.get(rand, rand)
        self.d[rand] = self.d.get(self.start, self.start)
        self.start +=1
        return divmod(ret, self.n_cols)
        
    def reset(self):
        """
        :rtype: void
        """
        self.start = 0
        self.d = {}

# Your Solution object will be instantiated and called as such:
# obj = Solution(n_rows, n_cols)
# param_1 = obj.flip()
# obj.reset()